# Latest Updates



| **Versión** | **Fecha** | **Update** | **Descripción** |
| :---: | :---: | :---: | :---: |
| v1 | 29/08/2025 | Implementación de Daily Budget | Se añadió la columna `Daily Budget Limit` en cada una de las hojas de eventos. |
| v2 | 05/09/2025 | Automatización de combos | En la página Sale Items se implementaron en la columna de eventos `New Combos` y `Old Combos` para su fácil mapeo durante la OC y se limpio esta misma columna de eventos obsoletos.
| v3 | 23/09/2025 | Automatización de Carga de Ítems | Se implementó el flujo para registrar ítems en eventos existentes y la posterior consulta automática de los resultados (tiendas/ítems exitosos) desde el Task Center.                                     |
| v3.1 | 26/09/2025 | Upload Items para todos lo eventos | Upload Items (Events Registration) soporta todos los tipos de eventos para su carga y registro histórico.    |
| v4 | 09/10/2025 | Mandar eventos solo por Gattaran | Se añadió la opción `Gattaran` en la columna `Invitation type` para mandar los eventos totalmente Offline (ni al BD ni al Rs). |
| v5 | 10/10/2025 | Combos multifuncionales | En la página de Sale Items se añadió en la columna de eventos la opción `Combos` para mandar eventos limitados a Combos sin incluirlos en Traffic Promotion como se hace en New Combos y Old Combos. |
| v6 | 10/10/2025 | Parametrización de rutas | El código es fácilmente transferible internamente gracias a la parametrización de rutas. |
| v7 | 02/12/2025 | Notebook globalizado | El Events Automation de `Acquisition` ahora es compatible con el notebook global. |
| v8 | 12/01/2025 | Mejoras y limpieza | Mejoras en las columnas relacionadas con la función `createPush`. |

# **Initialization**





## Requirements (dependencias)

**Copiar y pegar este código en la terminal cada 3 o 4 meses:**

`pip install requests pandas selenium webdriver-manager pillow google-auth google-auth-oauthlib google-auth-httplib2 gspread google-api-python-client`


## Team

In [1]:
global team
team = 'Acquisition' # Engagement, Acquisition, Acquisition CKA

## Libraries

In [2]:
import requests
import time
import os
import pandas as pd
import datetime as dt
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image
from datetime import datetime, timedelta
import random
import string
import math
from io import StringIO
from google.oauth2.credentials import Credentials
import gspread
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from google.oauth2 import service_account
from googleapiclient.discovery import build
from pathlib import Path

## Declaración de rutas

In [3]:
# DECLARACIÓN DE RUTAS

home_dir = Path.home()
documents_folder = None
if (home_dir / "Documentos").exists():
    documents_folder = home_dir / "Documentos"
elif (home_dir / "Documents").exists():
    documents_folder = home_dir / "Documents"
else:
    raise FileNotFoundError("No se encontró la carpeta 'Documentos' o 'Documents'.")

TICKET_PATH = documents_folder / "ticket.txt"
PW_PATH = documents_folder / "hi.txt"

READ_TOKEN_PATH = documents_folder / "token.json"

API_CONFIG_FOLDER = documents_folder / "api_config"
CREDENTIALS_PATH = API_CONFIG_FOLDER / "credentials.json"
WRITE_TOKEN_PATH = API_CONFIG_FOLDER / "token.json"

USERNAME_PATH = documents_folder / "username.txt"

if USERNAME_PATH.exists():
    username = USERNAME_PATH.read_text(encoding="utf-8").strip()
else:
    raise FileNotFoundError(f"No se encontró el archivo {USERNAME_PATH}")

print(documents_folder)
print(TICKET_PATH)
print(PW_PATH)
print(READ_TOKEN_PATH)
print(API_CONFIG_FOLDER)
print(CREDENTIALS_PATH)
print(WRITE_TOKEN_PATH)
print(USERNAME_PATH)

C:\Users\DiDi\Documents
C:\Users\DiDi\Documents\ticket.txt
C:\Users\DiDi\Documents\hi.txt
C:\Users\DiDi\Documents\token.json
C:\Users\DiDi\Documents\api_config
C:\Users\DiDi\Documents\api_config\credentials.json
C:\Users\DiDi\Documents\api_config\token.json
C:\Users\DiDi\Documents\username.txt


## API to read and write google sheets

In [4]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
creds = None

# Se intenta leer el token desde la ubicación principal "Documentos"
if READ_TOKEN_PATH.exists():
    creds = Credentials.from_authorized_user_file(READ_TOKEN_PATH, SCOPES)

if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        # Se leen las credenciales desde la subcarpeta "api_config"
        flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_PATH, SCOPES)
        creds = flow.run_local_server(port=0)

    # Se asegura que la carpeta "api_config" exista
    API_CONFIG_FOLDER.mkdir(exist_ok=True)

    # Se guarda el token nuevo o refrescado en la subcarpeta "api_config"
    with open(WRITE_TOKEN_PATH, 'w') as token:
        token.write(creds.to_json())

gc = gspread.authorize(creds)


## Authentication

In [5]:
#Ticket creation
def get_text_from_file(file_name):
    try:
        with open(file_name, 'r') as file:
            text= file.readline().strip()
        return text
    except FileNotFoundError:
        return "The file was not found."
    except Exception as e:
        return f"Error reading the file: {e}"


def getTicket(username,contra):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL','browser': 'ALL'})
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=chrome_options)
    driver.get('https://me.didiglobal.com/project/stargate-auth/html/login.html?redirect_uri=https%3A%2F%2Fmis-auth.didiglobal.com%2Fauth%2F%3Fjumpto%3Dhttps%253A%252F%252Fdilu.didi-food.com%252Findex%252Findex%252FcheckLogin%253Fjto%253DaHR0cHM6Ly9nYXR0YXJhbi5kaWRpLWZvb2QuY29tLw%253D%253D%26app_id%3D2200166%26callback_index%3D0')
    user=driver.find_element("xpath",'//*[@id="username"]')
    user.send_keys(username)
    password=driver.find_element("xpath",'//*[@id="password"]')
    password.send_keys(contra)
    driver.find_element("xpath",'//*[@id="submit"]/span').click()
    time.sleep(2)
    logs = str(driver.get_log("performance"))
    driver.quit()
    index = logs.find('TICKET=')
    if index != -1:
        result = logs[index + len('TICKET='):]
        semicolon_index = result.find(";")
        if semicolon_index != -1:
            ticket = result[:semicolon_index]
            return ticket
        else:
            return "; not found"
    else:
        return "Ticket not found"

def actualice_ticket():
    with open(TICKET_PATH, 'w') as archivo:
        archivo.write(getTicket(username,get_text_from_file(PW_PATH)))


def validate_ticket():
    error=''
    global response
    while error!=0:
        ticket= get_text_from_file(TICKET_PATH)
        form_to_validate_ticket= {
            "api": "spacex.msg.message.isNew",
            "country": "MX",
            "isCard": 0,
            "locale": "en-US",
            "msgChannel": 1,
            "referer": "https://gattaran.didi-food.com/v2/home",
            "signature": "",
            "ticket": ticket,
            "timestamp": "",
            "timezone": "America/Mexico_City",
            "uid": username
        }
        headers_to_validate_ticket={
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/113.0',
            'Accept': 'application/json',
            'Accept-Language': 'es-MX,es;q=0.8,en-US;q=0.5,en;q=0.3',
            'Accept-Encoding': 'gzip, deflate, br',
            'Content-Type': 'application/json',
            'Connection': 'keep-alive',
            'Sec-Fetch-Dest': 'empty',
            'Sec-Fetch-Mode': 'cors',
            'Sec-Fetch-Site': 'same-site',
        }
        #https://dilu.didi-food.com/gateway?
        response=requests.post('https://dilu.didi-food.com/gateway',json=form_to_validate_ticket,headers=headers_to_validate_ticket)
        error=response.json()['errno']
        if error==0:
            return ticket

        elif error==10000:
            actualice_ticket()
            ticket= get_text_from_file(TICKET_PATH)

In [6]:
# It is recommended to refresh this value periodically.
wsgig= "dd03-EqDIGGMtZCkl7NSp1cKP3bSmRsdi2RIR4nNwNjehRsdj5pxtIcXP3C5qubkj57kP6gJV2b1XuskVLvBz4mbw+bBjYbOmHvru1Dfx3CBlYgYh57kp1cbw+C9ruCYn"

## Headers

In [7]:
# These headers are necessary to communicate with https://dilu.didi-food.com/gateway?wsisg=
# When a form containing event details is submitted to this link, it processes the data,
# and responds with the unique ID assigned to the event.

headersEvent = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/113.0',
    'Accept': 'application/json',
    'Accept-Language': 'es-MX,es;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Content-Type': 'application/json',
    'Connection': 'keep-alive',
    'Sec-Fetch-Dest': 'empty',
    "operate-source": "script",
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
}


In [8]:
#These headers are necessary to communicate with https://dilu.didi-food.com/common?api=common.uploadPublicPic.
#When a file is sent to this link, it gets stored at a designated location,
# and the response provides a key that complements a link where the file is stored.
def FheadersImageCSV():
    headersImageCSV = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/113.0',
        'Accept': '*/*',
        'Accept-Language': 'es-MX,es;q=0.8,en-US;q=0.5,en;q=0.3',
        'Accept-Encoding': 'gzip, deflate, br',
        'ticket': ticket ,
        "operate-source": "script",
        'Connection': 'keep-alive',
    }
    return headersImageCSV

## Folders

In [9]:
def googleSheetLink(Team):
	if Team=='Engagement':
			return 'https://docs.google.com/spreadsheets/d/1t-Xfn3lRjGimtpXJYfgvKd-WYTUPY4VD2bjvMs_iTp0/edit#gid=0'
	elif Team=='CKA':
			return 'https://docs.google.com/spreadsheets/d/17DMTYB0Ue34vlz1tj36nUNEbCbsfduKkhFQmU1ImiAw/edit#gid=568495210'
	elif Team=='Acquisition':
			return 'https://docs.google.com/spreadsheets/d/1XuqsHdlUWJkfi8FPgNxoxOnWdi2S9oQs2OfsjDDktks/edit#gid=1309622116'
	elif Team=='Empowerement':
			return 'https://docs.google.com/spreadsheets/d/1QBnzLqicg2dXoeUtM8Std7HQeUEJJa7K1418c-hJOeQ/edit#gid=1309622116'
	elif Team== 'RExperience':
			return 'https://docs.google.com/spreadsheets/d/18zBiQQ7wCZ8cjO9c8Cn2-QpFpSplHXxwws4eZc5EbAM/edit#gid=1506897784'
	elif Team== 'Acq CKA':
		  return  'https://docs.google.com/spreadsheets/d/16IxoJ1K9PPItEuV6m9brotWPUCJrQ7-KUAxkJQsDkKg/edit?gid=33183766'

In [10]:
folderDriveImagesR=r'G:\Mi unidad\Imagenes'

## URLS necesarias

In [11]:
# This is the link where a POST request is sent to submit all the event information.
# and it returns the unique ID assigned to the created event.
urlToUploadEvent = 'https://dilu.didi-food.com/gateway?wsisg='+wsgig
#test
#urlToUploadEvent = 'https://sim060.rlab.sim.intra.xiaojukeji.com/dilu/gateway?wsgsig='+wsgig

#This link is used to make a POST request with a CSV file containing shop IDs or with a PNG image file
#and it returns the key that complements a link where the file has been stored.
urlToUpdloadFiles='https://dilu.didi-food.com/common?api=common.uploadPublicPic'
#test
#urlToUpdloadFiles='https://sim060.rlab.sim.intra.xiaojukeji.com/dilu/common?api=common.uploadPublicPic'

# Links where the files are stored
urlFormCSVImage='https://img0.didiglobal.com/static/soda_public/'#+key
urlFormCSVImageHTTP='http://img0.didiglobal.com/static/soda_public/'#+key

# Link to access the Google Sheets document containing events data.
sheets=['Sale Items','Delivery','SxSy','BxGy']#,'Sale Item','On Top Of','Delivery']


## Functions

### Function to post event information and receive event ID in response.

In [12]:
traceIDs=[]

def getEventID(datos):
	url=urlToUploadEvent
	response = requests.post(url, json=datos, headers=headersEvent)

	if response.status_code == 200:
			errmsg = response.json()['errmsg']

			if errmsg=='ok':
				data = response.json()['data']
				MerchantsPlanID = data['MerchantsPlanID']
				traceID = response.json()['traceId']
				print(f'traceID = {traceID}')
				traceIDs.append(traceID)

				print(MerchantsPlanID)
				return MerchantsPlanID

			else:
				print(response.json())
				return None
	else:
			print(f'Request Error:": {response.status_code}')
			return None


### Function to upload a file (CSV or PNG) and receive the key.

In [13]:
def csv_to_xlsx(input_csv):
    df = pd.read_csv(input_csv)
    df=df.astype(str)
    file_name = input_csv.split('.')[0]
    output_xlsx = file_name + '.xlsx'
    df.to_excel(output_xlsx, index=False, engine='openpyxl')

In [14]:
def redimensionarImagen(nameimagen):
    path_imagen=folderDriveImages +nameimagen
    try:
        imagen = Image.open(path_imagen)

        nueva_dimension = (520, 292)
        imagen_redimensionada = imagen.resize(nueva_dimension)
        nombre, extension = nameimagen.split(".")
        nuevo_nombre = folderDriveImagesR + f"{nombre} (CEL).{extension}"
        imagen_redimensionada.save(nuevo_nombre)

        nueva_dimension=(520,166)
        imagen_redimensionada = imagen.resize(nueva_dimension)
        nombre, extension = nameimagen.split(".")
        nuevo_nombre = folderDriveImagesR + f"{nombre} (PC).{extension}"
        imagen_redimensionada.save(nuevo_nombre)

        nueva_dimension=(588,330)
        imagen_redimensionada = imagen.resize(nueva_dimension)
        nombre, extension = nameimagen.split(".")
        nuevo_nombre = folderDriveImagesR + f"{nombre} (Ad).{extension}"
        imagen_redimensionada.save(nuevo_nombre)

    except Exception as e:
        print(f"Error: {e}")

In [15]:
def shopids_toInt(file):
    try:
        # 1. Leer asegurando que no haya columnas fantasma
        df = pd.read_csv(file, header=None).dropna()
        
        # 2. Convertir a numérico y asegurar que sean enteros
        df[0] = pd.to_numeric(df[0], errors='coerce').astype('Int64')
        df = df.dropna() # Eliminar cualquier fila que no fuera un número
        
        # 3. GUARDADO CRÍTICO: 
        # Forzamos encoding utf-8 y line_terminator para evitar caracteres extraños
        df.to_csv(file, index=False, header=False, encoding='utf-8', lineterminator='\n')
        
        global totalRs
        totalRs = len(df)
        print(f'file len : {totalRs}')

    except Exception as e:
        print(f"Error en shopids_toInt: {e}")

In [16]:
def sxsy_threshold_discounts(string):
    def tipo_y_valor(s): return (int(s[1:]), {'S': 4, 'O': 5}.get(s[0], 2)) if s[0].isalpha() else (int(s), 2)
    return [{
        "content": {
            "discount": d, "discountType": dt,
            "threshold": t, "thresholdType": tt
        },
        "type": 1
    } for par in string.split(',') for t, tt in [tipo_y_valor(par.split('-')[0].strip())] for d, dt in [tipo_y_valor(par.split('-')[1].strip())]]

In [17]:
def getKeyImageCSV(fileName, items=False):
  if items==True:
    folder = folderDriveTemplates

  elif fileName.endswith('.csv'):
    folder=folderDriveShopLists
    #csv_to_xlsx(folder + fileName)
    #fileName = fileName.split('.')[0]+ '.xlsx'
    #print(fileName)

  elif fileName.endswith('.png'):
    folder=''

  elif fileName.endswith('.xlsx'):
    folder=folderDriveShopLists

  shopids_toInt(folder / fileName)
  files = {
      'upfile': open(folder / fileName, 'rb')
    }



  response = requests.post(urlToUpdloadFiles, files=files, headers=headersImageCSV)
  if response.status_code == 200:
    errmsg = response.json()['errmsg']
    if errmsg == 'ok':
      print(response.json()['data']['key'])
      return  response.json()['data']['key']
    else:
      return errmsg

  else:
    print('Request error')

## Functions to set the event form variables

In [18]:
def timeZone(country):
  timezone_mapping = {
    'MX': 'Mexico_City',
    'CO': 'Bogota',
    'PE': 'Lima',
    'CR': 'Costa_Rica'
  }
  timezone = timezone_mapping.get(country, '')
  timezone='America/'+timezone
  return timezone

In [19]:
def getActType(promoType):
  event_mapping = {
    'Sale Items': 2,
    'On Top Of': 2,
    'BxGy': 4,
    'Delivery': 5,
    'SxSy' : 1
  }

  return event_mapping.get(promoType)

In [20]:
def invType(invitationType):
  invitationType_mapping = {
    'Offline': 2,
    'Online': 1,
  }
  return invitationType_mapping.get(invitationType, 'Incorrecto')

In [21]:
def getActSceneType(PromoScenario):
  promoScenario_mapping = {

      'Short Term': 16,
      'GP':15,
      'Ruta': 1,
      'Extra': 1,
      'Extra ST': 1,
      'Extra Flagship':1,
      'Else': 1 ,
      'Super Special': 2,
      'Auto Promo' : 3,
      'Churn Prevention': 4,
      'Churn Recovery' : 5,
      'SUP':6,
      'Flagship' : 7,
      'Week Of' :8,
      'New Combos' : 9,
      'Old Combos' : 9,
      'Combos' : 9,
      'KA Sports': 10,
      'Marketing': 11,
      'Promo Wow': 12,
			'SUP': 6,
			'SUP1': 6,
			'SUP2': 6,
			'ELC': 2,
			'Churn': 6,
			'Boost' : 1
  }

  return promoScenario_mapping.get(PromoScenario)

In [22]:
def EventLink(actType,rewardMode):
  host='gattaran.didi-food.com'
  #host='sim060.rlab.sim.intra.xiaojukeji.com'
  return f"https://{host}/v2/gtr_merchant-op/shopV2/signUp/add?actType={actType}&activityMode={rewardMode}"


In [23]:
def setDuration(startHour, endHour):
	if int(startHour)<int(endHour):
		duration=[
					{
				"duration": {
					"begin": f'{startHour}:00',
					"end": f'{endHour}:00'
					}
				}
			]

	elif int(startHour)>int(endHour):
		duration=[
				{
					"duration": {
						"begin": f'00:00',
						"end": f'{endHour}:00'
					}
				},
				{
					"duration": {
						"begin": f'{startHour}:00',
						"end": f'24:00'
					}
				}
			]
	return duration

In [24]:
def setDays(days,startHour,endHour, isNewCombo=0):
	startHour=startHour.zfill(2)
	endHour=endHour.zfill(2)
	duration=setDuration(startHour,endHour)

	if len(days)==7:
		periodSoldInfo= {
		  	"dayType": duration,
			  "soldInfoType": 1,
			  "weekType": None
		}


	else:
		days_dict = {}

		for day in days:
			days_dict[str(day)] = duration
		periodSoldInfo={
		  	"dayType": [],
			  "soldInfoType": 0,
			  "weekType": days_dict
		}

	return periodSoldInfo

In [25]:
def setCityID(Country):
  cityID_mapping = {
    'MX':[ 52090100],
    'PE': [51541300],
    'CR': [506030100],
    'CO':  [57380100]
    }
  return cityID_mapping.get(Country, 'Incorrecto')

In [26]:
def setCountyGroupID(Country):
  countyGroupID_mapping={
      'MX': [
			"520901000001",
			"521516000001",
			"521202000001",
			"520401000001",
			"522301000001",
			"520402000001",
			"522303000001",
			"520802000001",
			"521205000001",
			"523002000001",
			"520702000001",
			"523012000001",
			"520804000001",
			"520808000001",
			"522805000001",
			"523103000001",
			"523023000001",
			"520502000001",
			"521905000001",
			"522801000001",
			"522001000001",
			"523013000001",
			"520603000001",
			"520503000001",
			"522302000001",
			"523006000001",
			"522802000001",
			"520505000001",
			"520708000001",
			"522404000001",
			"522803000001",
			"520709000001",
			"522106000001",
			"520504000001",
			"520711000001",
			"522401000001",
			"523009000001",
			"522804000001",
			"522701000001",
			"523010000001",
			"520101000001",
			"521102000001",
			"522601000001",
			"520601000001",
			"521702000001",
			"522501000001",
			"521002000001",
			"520203000001",
			"521405000001",
			"522609000001",
			"522602000001",
			"521104000001",
			"520302000001",
			"521602000001",
			"521105000001",
			"520303000001",
			"522504000001",
			"521103000001",
			"520602000001",
			"522503000001",
			"520201000001",
			"521604000001",
			"522603000001",
			"522604000001",
			"521311000001",
			"522104000001",
			"522204000001",
			"521106000001",
			"522202000001",
			"521108000001",
			"521802000001",
			"520202000001",
			"522904000001",
			"521315000001",
			"521608000001",
			"521409000001",
			"523201000001",
			"521609000001"
		] ,
      'PE': [
          "515413000001",
          "513110000001",
          "514602000001",
          "513415000001",
          "514710000001",
          "514114000001",
          "514806000001"
        ],
      'CO':[
			"573801000001",
			"573301000001",
			"573901000001",
			"570101000001",
			"571901000001",
			"572601000001",
			"573701000001",
			"572101000001",
			"572401000001",
			"570501000001",
			"570401000001",
			"570701000001",
			"572201000001",
			"579999000001"
		] ,
      'CR': [
			"5060301000001",
			"5060501000001",
			"5060701000001",
			"5069999000001"
		],
  }
  return countyGroupID_mapping.get(Country)

In [27]:
def setShopFilter(fileName,country):

	key=getKeyImageCSV(fileName)

	form_len_file={
		"api": "tagcenter.crowd.CheckFile",
		"country": country,
		"entity": "shop",
		"file": "https://img0.didiglobal.com/static/soda_public/"+ key,
		"locale": "en-US",
		"ticket": ticket,
		"timezone": timeZone(country),
		"urlPath": "https://gattaran.didi-food.com/tagcenter/circleSelect/addCircleSelect?entity=shop"
	}

	global total_num
	global check5
	check5 = requests.post('https://dilu.didi-food.com/gateway?wsgsig='+wsgig,json=form_len_file, headers=headersEvent)
	total_num= check5.json()['data']['valid']
	if total_num==0:
		return 7



	create_tag_form={
		"api": "tagcenter.crowd.Upsert",
		"application": "promotions",
		"country": country,
		"creator": username,
		"crowd_name": fileName.strip('.csv')+'_'+''.join(random.choices(string.ascii_letters, k=3)),
		"crowd_rule": "{\"relation\":\"and\",\"rules\":[],\"citys\":[],\"areas\":[],\"channel\":0}",
		"data_source": "file",
		"entity": "shop",
		"file": "https://img0.didiglobal.com/static/soda_public/"+key,
		"file_name": fileName,
		"locale": "en-US",
		"ticket": ticket,
		"timezone": timeZone(country),
		"total_num": total_num,
		"update_type": 1,
		"urlPath": "https://gattaran.didi-food.com/tagcenter/circleSelect/addCircleSelect?entity=shop",
		"valid_time": f"{(datetime.today() + timedelta(days=3)).strftime('%Y-%m-%d')} 00:00:00"
	}
	response=requests.post('https://dilu.didi-food.com/gateway?wsgsig='+wsgig,json=create_tag_form,headers=headersEvent).json()

     
	if 'data' not in response or response['data'] is None:
		print("❌ Error en la API de DiDi:")
		print(response) # Esto te dirá el mensaje de error real (ej. "Sign error" o "Param error")
# ---------------------------------------

	return {
		"shopFilterType": 3,
		"tag": {
			"groupName": response['data'][0]['crowd_name'],
			"tagFileUrl": "https://img0.didiglobal.com/static/soda_public/"+key,
			"tagId": str(response['data'][0]['crowd_id']),
			"userTypeName": "shop"
		}
	}


In [28]:
def setSpecialItemRule(ACTtype,maxItems,minItems,DiscountType,discountRange,rBurnDiscount, event, itemType, restrict):

	itemType = 1 if event == 'Old Combos' or event == 'New Combos' or event == 'Combos' else itemType #1 combo, 0 single item, None all item types
	if itemType=='All':
		itemType=None
	elif itemType=='Combo':
		itemType=1
	#elif itemType=='':

	discount_mapping = {
    'Discount Range': 0,
	'Discount Tiers': 1,
    'Instant Discount': 2,
    'Upfront Price': 3
    }
	discountType=discount_mapping.get(DiscountType)
	if discountType==0:

		limitsDiscount = discountRange.split("-")
		maxDiscount=int(limitsDiscount[1])
		minDiscount=int(limitsDiscount[0])
		discountValueRange= {
			"max": maxDiscount,
			"min": minDiscount
			}
		discountValueList=[]
		discountT=0
		type=1

	elif discountType==1:
		discountT= 1
		type= 1
		discountValueRange= None
		valores = discountRange.split(",")
		discountValueList=[int(valor) for valor in valores]

	elif discountType==2:
		limitsDiscount = discountRange.split("-")
		maxDiscount=int(limitsDiscount[1])
		minDiscount=int(limitsDiscount[0])
		discountValueList=[]
		discountValueRange= {
								"max": maxDiscount,
								"min": minDiscount
							}
		discountT=None
		type=2

	elif discountType==3:
		discountT=1
		type=3
		discountValueRange= None
		valores = discountRange.split(",")
		discountValueList=[int(valor) for valor in valores]

	if ACTtype=='On top of':
		limitsDiscountRburn = rBurnDiscount.split("-")
		maxRburn=int(limitsDiscountRburn[1])
		minRburn=int(limitsDiscountRburn[0])
		itemPromoRangeValue={
							"max": maxRburn  ,
							"min": minRburn ,
						}

	else:
		itemPromoRangeValue=None

	specialItemRule=[
			{
				"checkItemPriceDay": 7 if (restrict =='Yes' or restrict == 7) else None,# none para no restringir
				"itemCrowdType": 0,
				"itemPromoRangeValue": itemPromoRangeValue ,
				"itemType": itemType, #1 combos, 0 single items, None all
				"picLimit": 0 if team=='Acquisition' else 1,
				"priceLimit": priceLimit,
				"rulePurposeType": 0,
				"rules": [
					{
						"content": {
							"discountType": discountT,
							"discountValue": 0,
							"discountValueList": discountValueList,
							"discountValueRange": discountValueRange
						},
						"type": type
					}
				],
				"selectItemNumRange": {
					"max": maxItems,
					"min": minItems
				}
			}
		]
	return specialItemRule

In [29]:
def setGetFDdiscount(spendxgety):
  valores = spendxgety.split(",")
  threshold=int(valores[0])
  discount=int(valores[1])

  return { "discountsLimit": discount, "usingThreshold": threshold}

In [30]:
def setFDRule(Disc, Thresh):
    def parse_range(val):
        if '-' in str(val):
            min_val, max_val = map(int, val.split('-'))
            return {"min": min_val, "max": max_val}
        return {"min": 0, "max": 0}

    def get_type(val_str):
        if val_str.startswith('S'):
            return 4
        elif val_str.startswith('O'):
            return 5
        elif '-' in val_str:
            return 3
        elif int(val_str) == 0:
            return 1
        else:
            return 2

    def get_fixed(val_str, val_type):
        if val_type in (4, 5):
            return int(val_str[1:])
        elif val_type == 3:
            return 0
        else:
            return int(val_str)

    rules = []
    for d, t in zip(Disc, Thresh):
        d_str, t_str = str(d), str(t)

        discount_type = get_type(d_str)
        threshold_type = get_type(t_str)

        rule = {
            "content": {
                "discountsLimit": 0,
                "discountsLimitType": discount_type,
                "fixedDiscountsLimit": get_fixed(d_str, discount_type),
                "fixedUsingThreshold": get_fixed(t_str, threshold_type),
                "rangeDiscountsLimit": parse_range(d_str),
                "rangeUsingThreshold": parse_range(t_str),
                "usingThreshold": 0,
                "usingThresholdType": threshold_type
            },
            "type": 1
        }
        rules.append(rule)

    return [{"rulePurposeType": 0, "rules": rules}]

In [31]:
def setSxSyDiscount(sxsy):
  valores=sxsy.split(',')
  threshold=int(valores[0])
  discount=int(valores[1])
  return  { "discount": discount,"threshold": threshold	}

In [32]:
def presentPlatformList(ONOff):
    if type(ONOff) is not list:
        if ONOff=='Offline':
            return [2,4]
        elif ONOff=='Online':
            return [1]
        elif ONOff == 'On/Off':
            return [1,2,4]
        elif ONOff == 'Gattaran':
            return [4]
    else:
        lista=[]
        if ONOff[0]=='TRUE':
            lista.append(1)
        if ONOff[1]=='TRUE':
            lista.append(2)
        if ONOff[2]=='TRUE':
            lista.append(4)
        return lista

In [33]:
def createPush(adDescription,adText,nameImage):
	pathImage=folderDriveImages +nameImage
	nombre, extension = nameImage.split(".")
	nameImageCEL = f"{nombre} (CEL).{extension}"
	nameImagePC = f"{nombre} (PC).{extension}"
	nameImageAd= f"{nombre} (Ad).{extension}"
	if not os.path.exists(folderDriveImagesR + nameImageCEL):
		redimensionarImagen(nameImage)
	imageKeyPC =getKeyImageCSV(folderDriveImagesR + nameImagePC)
	imageKeyCEL=getKeyImageCSV(folderDriveImagesR + nameImageCEL)
	imageKeyAd=getKeyImageCSV(folderDriveImagesR + nameImageAd)
	if team == 'Acquisition':
		channels=[
			"whatsApp",
			"home",
			"email",
			"pop"
		]
	else:
		channels=[
			"home",
			"pop"
		]

	PUSH= {
		"bannerImg": [
			{
				"filename": nameImage,
				"key": imageKeyAd,
				"name": nameImage,
				"status": "success",
				"url": "http://img0.didiglobal.com/static/soda_public/" + imageKeyAd,
				"urlHttps": "https://img0.didiglobal.com/static/soda_public/" + imageKeyAd
			}
		],
		"introText": adDescription,
		"popImgs":  {
					"other": {
						"name": nameImage,
						"url": "https://img0.didiglobal.com/static/soda_public/"+imageKeyPC
					},
					"phone": {
						"name": nameImage,
						"url": "https://img0.didiglobal.com/static/soda_public/"+imageKeyCEL
					}
				},
		"pushChannel": channels,
		"pushText": adText
	}
	return PUSH

### Functions to read and write google sheets

In [34]:
def writeEventID(row_number, value,sheetName,flag):
    sh = gc.open_by_url(urlGoogleSheet).worksheet(sheetName)
    sh.update(f'D{row_number}', [[value]])
    sh.update(f'C{row_number}', [[f'Rs: {total_num} / {totalRs}']])
    sh.update(f'B{row_number}', [[flag]])
    print(f'Rs: {total_num} / {totalRs} \n ')

In [35]:
def SheetToDataframe(url,sheetName,offset=0):
    sh = gc.open_by_url(url).worksheet(sheetName).get_all_values()
    df = pd.DataFrame(sh[offset + 1:], columns=sh[offset])
    df.index.name = 'rowNumber'
    df.reset_index(inplace=True)
    return df

In [36]:
def setSubsidyBudget(rewardRatio,maxRewardperItem,budgetLimit, dailyBudget=0):#,disc):
	if budgetLimit=='' or budgetLimit== 0 or budgetLimit== '0':
		budgetLimit=9999999999
	if dailyBudget=='' or dailyBudget== 0 or dailyBudget== '0':
		dailyBudget=0
	maxRewardperItem=int(maxRewardperItem)
	budgetLimit=int(budgetLimit)
	dailyBudget=int(dailyBudget)

	rewardRatio = str(rewardRatio)
	if not  '-' in rewardRatio:
		print(rewardRatio)
		rewardRatio=int(rewardRatio)
		print(rewardRatio)
		subsidyInfo={
				"actPlatformBudgetLimit": 9999999999 if team == 'Acquisition' else budgetLimit ,
				"actShopBudgetLimit": 0 if team != 'Acquisition' else budgetLimit,
				"actShopBudgetThreshold": {
					"enable": False,
					"threshold": 0
				},
				"actSubsidyLevelRule": {} ,
				"actSubsidyLimit": 18000 if country=='CO' and team != 'Acq CKA' else 0,
				"actSharePlatformBudgetPeriodDayLimit": dailyBudget,
				"actSubsidyList": [
				{
					"subsidyAmount": rewardRatio
				}
				],
				"actSubsidyRange": {
					"max": 0,
					"min": 0
				},
				"actSubsidyTargetType": 0 ,
				"factorType": 0,
				"isCopy": False,
				"subsidyAmount": rewardRatio
			}

	else:
		subs = rewardRatio.split('-')
		subs = [int(s) for s in subs]
		subsidyInfo = {
			"actPlatformBudgetLimit": 9999999999 if team == 'Acquisition' else budgetLimit,
			"actSharePlatformBudgetPeriodDayLimit": dailyBudget,
			"actShopBudgetLimit": 0,
			"actShopBudgetThreshold": {
				"enable": False,
				"threshold": 0
			},
			"actSubsidyLevelRule": {},
			"actSubsidyLimit":  18000 if country=='CO' and team != 'Acq CKA' else 0,
			"actSubsidyList": [
				{
					"subsidyAmount": 0
				}
			],
			"actSubsidyRange": {
				"max": subs[1],
				"min": subs[0]
			},
			"actSubsidyTargetType": 5,
			"factorType": 0,
			"isCopy": False,
			"subsidyAmount": 0
		}
	return subsidyInfo

In [37]:
def protectRow(ROW,sheetName):
    emailAddress = f"{username}@didi-labs.com"
    spreadsheet=gc.open_by_url(urlGoogleSheet)
    sheetId=spreadsheet.worksheet(sheetName).id

    body = {
    "requests": [
        {
            "addProtectedRange": {
                "protectedRange": {
                    "range": {
                        "sheetId": sheetId,
                        "startRowIndex": ROW-1,
                        "endRowIndex": ROW,
                        "startColumnIndex": 1,
                        "endColumnIndex": 99
                    },

                    "editors": {
                        "domainUsersCanEdit": False,
                        "users": [emailAddress]
                    },
                    "warningOnly": False
                    }
                }
            }
        ]
    }
    spreadsheet.batch_update(body)

In [38]:
def createPriceLimit(MinPrice,MaxPrice):
	global priceLimit
	if MinPrice=='' and MaxPrice=='':
		priceLimit=None
	else:
		priceLimit= {
					"max": int(MaxPrice),
					"min": int(MinPrice)
				}



In [39]:
user_dict={"All":0,
 "New":1,
 "Recurrent":2

}

### Function to create form

In [40]:
def createForm(event,dataList):
	rewardMode='Original Price Mode'
	threshold=discount=minItems=maxItems=minPrice=maxPrice=0
	global country
	if event=='SxSy':
		if team=='Engagement':
			country,eventReviewer,OnOff,Event,car,actName,internalActName,diferenciador,\
			registrationStartDate,registrationEndDate,actStartDate,actEndDate,\
			mon,tues,wed,thurs,fri,sat,sun,\
			startHour,endHour,\
			thr_disc,rewardRatio,usertype,budgetLimit, dailyBudget,\
			nameFileCSV,nameFileImage,adDescription,adText=dataList
		elif team=='CKA':
			country,eventReviewer,B_APP,BD_APP,Gattaran_brt,Event,actName,internalActName,diferenciador,\
			registrationStartDate,registrationEndDate,actStartDate,actEndDate,\
			mon,tues,wed,thurs,fri,sat,sun,\
			startHour,endHour,\
			threshold,discount,rewardRatio,budgetLimit,\
			nameFileCSV,nameFileImage,adDescription,adText=dataList
			OnOff=[B_APP,BD_APP,Gattaran_brt]
		else:
			country,eventReviewer,B_APP,BD_APP,Gattaran_brt,Event,actName,internalActName,diferenciador,\
			registrationStartDate,registrationEndDate,actStartDate,actEndDate,\
			mon,tues,wed,thurs,fri,sat,sun,\
			startHour,endHour,\
			threshold,discount,rewardRatio,budgetLimit,\
			nameFileCSV,nameFileImage,adDescription,adText=dataList
			OnOff=[B_APP,BD_APP,Gattaran_brt]

	elif event=='BxGy':
		if team=='Engagement':
			country,eventReviewer,OnOff,Event,car,actName,internalActName,diferenciador,\
			registrationStartDate,registrationEndDate,actStartDate,actEndDate,\
			mon,tues,wed,thurs,fri,sat,sun,\
			startHour,endHour,\
			discountType,rewardRatio,usertype,budgetLimit, dailyBudget,\
			nameFileCSV,nameFileImage,adDescription,adText,minItems,maxItems,minPrice,maxPrice=dataList
		elif team=='CKA':
			country,eventReviewer,B_APP,BD_APP,Gattaran_brt,Event,actName,internalActName,diferenciador,\
			registrationStartDate,registrationEndDate,actStartDate,actEndDate,\
			mon,tues,wed,thurs,fri,sat,sun,\
			startHour,endHour,\
			discountType,rewardRatio,budgetLimit,\
			nameFileCSV,nameFileImage,adDescription,adText,minItems,maxItems,minPrice,maxPrice=dataList
			OnOff=[B_APP,BD_APP,Gattaran_brt]
		else:
			country,eventReviewer,B_APP,BD_APP,Gattaran_brt,Event,actName,internalActName,diferenciador,\
			registrationStartDate,registrationEndDate,actStartDate,actEndDate,\
			mon,tues,wed,thurs,fri,sat,sun,\
			startHour,endHour,\
			discountType,rewardRatio,budgetLimit,\
			nameFileCSV,nameFileImage,adDescription,adText,minItems,maxItems,minPrice,maxPrice=dataList
			OnOff=[B_APP,BD_APP,Gattaran_brt]

	elif event=='Delivery':
		if team=='Engagement':
			country,eventReviewer,OnOff,Event,car,actName,internalActName,diferenciador,\
			registrationStartDate,registrationEndDate,actStartDate,actEndDate,\
			mon,tues,wed,thurs,fri,sat,sun,\
			startHour,endHour,\
			threshold,discount,rewardRatio,usertype,budgetLimit, dailyBudget,\
			nameFileCSV,nameFileImage,adDescription,adText=dataList
		elif team=='CKA':
			country,eventReviewer,B_APP,BD_APP,Gattaran_brt,Event,actName,internalActName,diferenciador,\
			registrationStartDate,registrationEndDate,actStartDate,actEndDate,\
			mon,tues,wed,thurs,fri,sat,sun,\
			startHour,endHour,\
			threshold,discount,rewardRatio,budgetLimit,\
			nameFileCSV,nameFileImage,adDescription,adText=dataList
			OnOff=[B_APP,BD_APP,Gattaran_brt]
		else:
			country,eventReviewer,B_APP,BD_APP,Gattaran_brt,Event,actName,internalActName,diferenciador,\
			registrationStartDate,registrationEndDate,actStartDate,actEndDate,\
			mon,tues,wed,thurs,fri,sat,sun,\
			startHour,endHour,\
			threshold,discount,rewardRatio,budgetLimit,\
			nameFileCSV,nameFileImage,adDescription,adText=dataList
			OnOff=[B_APP,BD_APP,Gattaran_brt]

		discount=discount.split(',')
		#discount = [int(x) for x in discount]
		threshold=threshold.split(',')
		#threshold = [int(x ) for x in threshold]
		if len(threshold)!=len(discount):
			return 5

	elif event== 'Sale Items':
		if team=='Engagement':
			country,eventReviewer,OnOff,Event,car,actName,internalActName,diferenciador,\
			registrationStartDate,registrationEndDate,actStartDate,actEndDate\
			,mon,tues,wed,thurs,fri,sat,sun,\
			startHour,endHour,\
			discountType,discounts,rewardMode,rburnDiscount,rewardRatio,usertype, budgetLimit, dailyBudget,\
			nameFileCSV,nameFileImage,adDescription,adText,minItems,maxItems,minPrice,maxPrice=dataList
			itemType=None
			restrictRaisePrices=None
		elif team=='CKA':
			country,eventReviewer,B_APP,BD_APP,Gattaran_brt,Event,actName,internalActName,diferenciador,\
			registrationStartDate,registrationEndDate,actStartDate,actEndDate,\
			mon,tues,wed,thurs,fri,sat,sun,\
			startHour,endHour,\
			discountType,discounts,rewardMode,rburnDiscount,rewardRatio, budgetLimit,\
			nameFileCSV,nameFileImage,adDescription,adText,itemType,restrictRaisePrices,minItems,maxItems,minPrice,maxPrice=dataList
			OnOff=[B_APP,BD_APP,Gattaran_brt]
		else:
			country,eventReviewer,B_APP,BD_APP,Gattaran_brt,Event,actName,internalActName,diferenciador,\
			registrationStartDate,registrationEndDate,actStartDate,actEndDate,\
			mon,tues,wed,thurs,fri,sat,sun,\
			startHour,endHour,\
			discountType,discounts,rewardMode,rburnDiscount,rewardRatio, budgetLimit,\
			nameFileCSV,nameFileImage,adDescription,adText,itemType,restrictRaisePrices,minItems,maxItems,minPrice,maxPrice=dataList
			OnOff=[B_APP,BD_APP,Gattaran_brt]


		if country=='CO' and team!='Acq CKA':
			maxRewardperItem=18000
		else:
			maxRewardperItem=0

		subsiInfo=setSubsidyBudget(rewardRatio,maxRewardperItem,budgetLimit, dailyBudget if team == 'Engagement' else 0)#,discounts)
		if subsiInfo==6:
			return 6

	if country=='CO' and team!='Acq CKA':
		maxRewardperItem=18000
	else:
		maxRewardperItem=0
	if team=='Acquisition':
		internalActName += '_' + registrationStartDate

	weekDays=[i+1 for i, dia in enumerate([mon, tues, wed, thurs, fri, sat, sun]) if dia == 'TRUE']
	#Excepciones
	if OnOff=='Online' or OnOff[0]=='TRUE' or OnOff=='On/Off':
		if nameFileImage=='':
			return 0
		if adText=='':
			return 1
		if adDescription=='':
			return 2
	if len(actName)>30:
		return 3
	if nameFileCSV=='':
		return 4

	if (OnOff=='Online' or OnOff[0]=='TRUE' or OnOff=='On/Off') and '-' in rewardRatio:
		return 9


	if country=='CDMX':
		country='MX'

	budgetLimit = budgetLimit.replace(',', '')
	if budgetLimit=='':
		budgetLimit=99999999999


	if event!= 'Delivery':
		if not ',' in rewardRatio and not '-' in rewardRatio :
			rewardRatio=int(rewardRatio)
		minItems,maxItems,budgetLimit,discount,threshold= [int(x) for x in [minItems,maxItems,budgetLimit,discount,threshold]]
	else:
		# if not ',' in rewardRatio and not '-' in rewardRatio :
		# 	rewardRatio=int(rewardRatio)
		minItems,maxItems,budgetLimit= [int(x) for x in [minItems,maxItems,budgetLimit]]

	if '.csv' not in nameFileCSV:
		nameFileCSV+='.csv'
	if '.png' not in nameFileImage:
		nameFileImage+='.png'
	shopFilter=setShopFilter(nameFileCSV,country)
	if shopFilter==7:
		return 7
	#if totalRs>20000:
	#	return 8
	#keyCSV=getKeyImageCSV(nameFileCSV)

	promoType=event

	presentList=presentPlatformList(OnOff)

	if  diferenciador!='':
		internalActName=internalActName+'_'+diferenciador

	actType=getActType(promoType)

	if rewardMode=='On top of':
		activityMode=1
	else:
		activityMode=0

	actSceneType=getActSceneType(Event)
	#merchantsPlanMode=invType(OnOff)

	if country=='CO' and team!='Acq CKA':
		maxRewardperItem=18000

	else:
		maxRewardperItem=0

	if (adDescription != "" or adText != "") and (OnOff=='Offline' or OnOff[0]=='FALSE'):
		push= {
				"introText": adDescription if adDescription != "" else "-",
				"popImgs": None,
				"pushChannel": [],
				"pushText": adText if adText != "" else "-"
		}

	elif OnOff=='Offline' or OnOff[0]=='FALSE' or OnOff=='Gattaran':
			push= {
				"introText": "-",
				"popImgs": None,
				"pushChannel": [],
				"pushText": "-"
		}

	elif OnOff=='Online' or OnOff=='On/Off' or OnOff[0]=='TRUE':
			push=createPush(adDescription,adText,nameFileImage)

	try:
		usertype=user_dict[usertype]
	except NameError:
		pass
	createPriceLimit(minPrice,maxPrice)
	if event=='Sale Items' and rewardMode=='Original Price Mode':
		datos={
			"actInfo": {
				"actDays": 30 if Event == 'New Combos' else 0,
				"actEndTime": "" if Event == 'New Combos' else actEndDate + " 23:59:59" ,
				"actName": actName ,
				"actStartTime": "" if Event == 'New Combos' else actStartDate + " 00:00:00" ,
				"actTimeType": 1 ,
				"actType": actType ,
				"internalActName": internalActName ,

				"periodSoldInfo": setDays(weekDays,startHour,endHour) ,
				"specialItemRule": setSpecialItemRule(rewardMode,maxItems,minItems,discountType,discounts,rburnDiscount,Event,itemType,restrictRaisePrices),
				"userType": 0 if team == 'Acquisition' else usertype
			},
			"api": "marketing-mis.marketing.merchantsPlan.create",
			"country": country ,
			"locale": "en-US",
			"merchantsPlanInfo": {
				"actFlowType": 1 if Event == 'New Combos' or Event == 'Old Combos' else 0,
				"activityMode": activityMode ,
				"actSceneType": actSceneType ,
				"auditUserList": [
					eventReviewer
				],
				"cityId": setCityID(country),
				"countyGroupIDs": setCountyGroupID(country),
				"joinLimitType": 0,
				"merchantsPlanEndTime": registrationEndDate + " 23:58:59" ,
				#"merchantsPlanMode": merchantsPlanMode,
				"merchantsPlanStartTime": registrationStartDate + " 00:00:00" ,
				"merchantsPlanType": 2 if Event == 'New Combos' else 1,
				"presentPlatformList":presentList,
				"radioFlowType": 0,
				"recommend": 0
			},
			#"player": "nevermore",
			"push": push,
			"referer": EventLink(actType,activityMode) ,
			"shopFilter": shopFilter,
			"subsidyBudget": subsiInfo,
			"ticket": ticket ,
			"operate-source": "script",
			"timezone": timeZone(country) ,
			"urlPath": EventLink(actType,activityMode)
		}
	#####################################################################################################################################
	elif event=='Sale Items' and rewardMode=='On top of':
		datos={
				"actInfo": {
					"actDays": 30 if Event == 'New Combos' else 0,
					"actEndTime": "" if Event == 'New Combos' else actEndDate + " 23:59:59" ,
					"actName": actName ,
					"actStartTime": "" if Event == 'New Combos' else actStartDate + " 00:00:00" ,
					"actTimeType": 1,
					"actType": actType ,
					"internalActName": internalActName ,
					"periodSoldInfo": setDays(weekDays,startHour,endHour, 1 if Event == 'New Combos' else 0) ,
					"specialItemRule": setSpecialItemRule(rewardMode,maxItems,minItems,discountType,discounts,rburnDiscount,Event,itemType,restrictRaisePrices)
				},
				"api": "marketing-mis.marketing.merchantsPlan.create",
				"country": country,
				"locale": "en-US",
				"merchantsPlanInfo": {
					"actFlowType": 1 if actSceneType == 9 else 0,
					"activityMode": activityMode ,
					"actSceneType": actSceneType ,
					"auditUserList": [
						eventReviewer
					],
					"cityId": setCityID(country),
					"countyGroupIDs":setCountyGroupID(country),
					"joinLimitType": 0,
					"merchantsPlanEndTime": registrationEndDate + " 23:58:59" ,
					#"merchantsPlanMode": merchantsPlanMode,
					"merchantsPlanStartTime": registrationStartDate + " 00:00:00" ,
					"merchantsPlanType": 2 if Event == 'New Combos' else 1,
					"presentPlatformList":presentList,
					"radioFlowType": 0,
					"recommend": 0
				},
				#"player": "nevermore",
				"push": push,
				"referer": EventLink(actType,activityMode),
				"shopFilter": shopFilter,
				"subsidyBudget": {
					"actPlatformBudgetLimit": 999999999999 if team == 'Acquisition' else budgetLimit,
					"actShopBudgetLimit": 0 if team != 'Acquisition' else budgetLimit,
					"actShopBudgetThreshold": {
						"enable": False,
						"threshold": 0
					},
					"actSubsidyLevelRule": {},
					"actSubsidyLimit": maxRewardperItem,
					"actSubsidyList": [
						{
							"subsidyAmount": rewardRatio
						}
					],
					"actSubsidyTargetType": 0,
					"factorType": 0,
					"subsidyAmount": rewardRatio
				},
				"ticket": ticket,
				"operate-source": "script",
				"timezone": timeZone(country),
				"urlPath": EventLink(actType,activityMode)
			}

	################################################################################################################################################
	elif event=='Delivery':


		datos={
			"actInfo": {
				"actDays": 0,
				"actEndTime": actEndDate + " 23:59:59" ,
				"actName": actName,
				"actStartTime":  actStartDate + " 00:00:00" ,
				"actTimeType": 1,
				"actType": actType ,
				"freeDeliveryRule": setFDRule(discount,threshold),
				"internalActName": internalActName,
				"periodSoldInfo": setDays(weekDays,startHour,endHour),
				"userType":usertype
			},
			"api": "marketing-mis.marketing.merchantsPlan.create",
			"country": country,
			"locale": "en-US",
			"merchantsPlanInfo": {
				"actFlowType": 0,
				"activityMode": activityMode,
				"actSceneType": actSceneType,
				"auditUserList": [
					eventReviewer
				],
				"cityId": setCityID(country),
				"countyGroupIDs": setCountyGroupID(country),
				"joinLimitType": 0,
				"merchantsPlanEndTime": registrationEndDate + " 23:58:59" ,
				#"merchantsPlanMode": merchantsPlanMode  ,
				"merchantsPlanStartTime": registrationStartDate + " 00:00:00" ,
				"merchantsPlanType": 1,
				"presentPlatformList":presentList,
				"radioFlowType": 0,
				"recommend": 0
			},
			#"player": "nevermore",
			"push": push,
			"referer": EventLink(actType,activityMode),
			"shopFilter": shopFilter,
			"subsidyBudget": setSubsidyBudget(rewardRatio,maxRewardperItem,budgetLimit, dailyBudget),
			"ticket": ticket,
			"operate-source": "script",
			"timezone": timeZone(country),
			"urlPath": EventLink(actType,activityMode)
		}
	#####################################################################################################################################################
	########################################################################################################################################
	elif event=='BxGy':
		if discountType=='Buy one get one free':
			buyNum=1
			getNum=2
		elif discountType=='Buy two get one free':
			buyNum=2
			getNum=3
		elif discountType=='Buy three get one free':
			buyNum=3
			getNum=4

		datos=  {
			"actInfo": {
				"actDays": 0,
				"actEndTime": actEndDate + " 23:59:59",
				"actName": actName ,
				"actStartTime": actStartDate + " 00:00:00" ,
				"actTimeType": 1,
				"actType": actType ,
				"buyGiftsRule": [
					{
						"checkItemPriceDay": None if team == "Engagement" else 7,
						"itemCrowdType": 0,
						"itemPromoRangeValue": None,
						"picLimit": 1,
						"priceLimit": priceLimit,
						"rulePurposeType": 0,
						"rules": [
							{
								"content": {
									"buyNum": buyNum,
									"getNum": getNum
								},
								"type": 1
							}
						],
						"selectItemNumRange": {
							"max": maxItems,
							"min": minItems
						}
					}
				],
				"internalActName": internalActName,
				"periodSoldInfo": setDays(weekDays,startHour,endHour),
				"userType":usertype
			},
			"api": "marketing-mis.marketing.merchantsPlan.create",
			"country": country ,
			"locale": "en-US",
			"merchantsPlanInfo": {
				"actFlowType": 0,
				"activityMode": activityMode,
				"actSceneType": actSceneType,
				"auditUserList": [
					eventReviewer
				],
				"cityId": setCityID(country),
				"countyGroupIDs": setCountyGroupID(country),
				"joinLimitType": 0,
				"merchantsPlanEndTime": registrationEndDate + " 23:58:59" ,
				#"merchantsPlanMode": merchantsPlanMode,
				"merchantsPlanStartTime": registrationStartDate + " 00:00:00",
				"merchantsPlanType": 1,
				"presentPlatformList":presentList,
				"radioFlowType": 0,
				"recommend": 0
			},
			#"player": "nevermore",
			"push": push,
			"referer": EventLink(actType,activityMode),
			"shopFilter": shopFilter,
			"subsidyBudget": setSubsidyBudget(rewardRatio,maxRewardperItem,budgetLimit, dailyBudget),
			"ticket": ticket,
			"operate-source": "script",
			"timezone": timeZone(country),
			"urlPath": EventLink(actType,activityMode)
		}
	elif event=='SxSy':
		datos={
			"actInfo": {
				"actDays": 0,
				"actEndTime": actEndDate + " 23:59:59",
				"actName": actName,
				"actStartTime": actStartDate + " 00:00:00" ,
				"actTimeType": 1,
				"actType": actType,
				"internalActName": internalActName,
				"periodSoldInfo": setDays(weekDays,startHour,endHour),
				"reductionRule": [
					{
						"rulePurposeType": 0,
						"rules": sxsy_threshold_discounts(thr_disc)
					}
				],
				"userType":usertype
			},
			"api": "marketing-mis.marketing.merchantsPlan.create",
			"country": country,
			"locale": "en-US",
			"merchantsPlanInfo": {
				"actFlowType": 0,
				"activityMode": activityMode,
				"actSceneType": actSceneType,
				"auditUserList": [
					eventReviewer
				],
				"cityId": setCityID(country),
				"countyGroupIDs": setCountyGroupID(country),
				"joinLimitType": 0,
				"merchantsPlanEndTime": registrationEndDate + " 23:58:59" ,
				#"merchantsPlanMode": merchantsPlanMode,
				"merchantsPlanStartTime": registrationStartDate + " 00:00:00",
				"merchantsPlanType": 1,
				"presentPlatformList":presentList,
				"radioFlowType": 0,
				"recommend": 0
			},
			#"player": "nevermore",
			"push": push,
			"referer": EventLink(actType,activityMode),
			"shopFilter": shopFilter,

		"subsidyBudget": setSubsidyBudget(rewardRatio,maxRewardperItem,budgetLimit, dailyBudget),
			"ticket": ticket,
			"operate-source": "script",
			"timezone": timeZone(country),
			"urlPath": EventLink(actType,activityMode)
		}
	return datos

### Function to upload events

In [41]:
errors_list=['Missing Image','Missing Ad Text', 'Missing Ad Description','Act name cannot exceed 30 characters','Missing Rs list','Thresholds len and discounts len are different','Rewards len and discounts len are different','Wrong file csv','CSV File exceeds 20k Rs','Rango de subsidio no disponible Online']

In [42]:
def defineFoldersAndGSlink(TEAM):
    print(f'-------------{TEAM}-------------')

    base = Path("G:/")

    # Detectar automáticamente nombres de carpetas según idioma
    shared = None
    mydrive = None

    for folder in base.iterdir():
        name = folder.name.lower()
        if "shared" in name or "unidades" in name:
            shared = folder
        elif "my drive" in name or "mi unidad" in name:
            mydrive = folder

    if shared is None or mydrive is None:
        raise FileNotFoundError("No se encontraron las carpetas de Google Drive ('Shared drives' o 'My Drive').")

    global folderDriveShopLists
    global folderDriveImages
    global folderDriveTemplates
    global urlGoogleSheet
    global folderDriveImagesR

    folderDriveShopLists = shared / f"Analytics_RCentral/Promos Automation/{TEAM}/Events/XLSX"
    folderDriveImages = shared / f"Analytics_RCentral/Promos Automation/{TEAM}/Events/Images"
    folderDriveTemplates = shared / f"Analytics_RCentral/Promos Automation/{TEAM}/Events/Templates"
    folderDriveImagesR = mydrive / "Images"
    urlGoogleSheet = googleSheetLink(TEAM)

    print("Shop Lists:", folderDriveShopLists)
    print("Images:", folderDriveImages)
    print("Templates:", folderDriveTemplates)
    print("Images (My Drive):", folderDriveImagesR)


In [43]:
def uploadEvent(team):
			defineFoldersAndGSlink(team)
			for event in sheets:
					events=SheetToDataframe(urlGoogleSheet,event)
					global check
					check=events
					for index, row in events.iterrows():
							readyToSent = row['Ready to Send']
							ID= row['ID']
							Sent = row['Sent']

							if readyToSent=='TRUE' and ID=='' and Sent=='FALSE': #check if an event is ready to be uploaded
									global ticket
									ticket=validate_ticket()
									global headersImageCSV
									headersImageCSV=FheadersImageCSV()
									row['rowNumber']+=2
									dataList=row.tolist()[7:]
									print(dataList)
									form=createForm(event,dataList)
									if form in range(len(errors_list)):
											print(errors_list[form])
											writeEventID(row['rowNumber'],errors_list[form],event,False)
									else:
											print(form)
											writeEventID(row['rowNumber'],getEventID(form),event, True)
											protectRow(row['rowNumber'],event)
									#time.sleep(random.randint(240, 420))
			time.sleep(60)

# Create Events

In [ ]:
while True:
    uploadEvent(team)
    time.sleep(60) # Each minute upload the ready events


-------------Acquisition-------------
Shop Lists: G:\Shared drives\Analytics_RCentral\Promos Automation\Acquisition\Events\XLSX
Images: G:\Shared drives\Analytics_RCentral\Promos Automation\Acquisition\Events\Images
Templates: G:\Shared drives\Analytics_RCentral\Promos Automation\Acquisition\Events\Templates
Images (My Drive): G:\My Drive\Images
['MX', 'marcoangeleslopez', 'FALSE', 'TRUE', 'TRUE', 'SUP1', '¡Bienvenido a DiDi Food!', 'SUP_1_Video_PT2', '', '2026-01-09', '2026-02-05', '2026-01-09', '2026-02-06', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', '0', '24', 'Discount Tiers', '35', 'Original Price Mode', '', '100', '15000', 'inv_video_2', '', '', '', 'All', 'No', '1', '10', '1', '350']
100
100
file len : 2
do1_ErLB8FEkWGx1lpcZEwRz
{'actInfo': {'actDays': 0, 'actEndTime': '2026-02-06 23:59:59', 'actName': '¡Bienvenido a DiDi Food!', 'actStartTime': '2026-01-09 00:00:00', 'actTimeType': 1, 'actType': 2, 'internalActName': 'SUP_1_Video_PT2_2026-01-09', 'periodSoldInfo': 

C:\Users\DiDi\AppData\Local\Temp\ipykernel_35944\1032716371.py:3: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sh.update(f'D{row_number}', [[value]])
C:\Users\DiDi\AppData\Local\Temp\ipykernel_35944\1032716371.py:4: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sh.update(f'C{row_number}', [[f'Rs: {total_num} / {totalRs}']])
C:\Users\DiDi\AppData\Local\Temp\ipykernel_35944\1032716371.py:5: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sh.update(f'B{row_number}', [[flag]])


Rs: 1 / 2 
 


# Pausar, Detener Eventos

In [1]:
global ticket
ticket=validate_ticket()
global headersImageCSV
headersImageCSV=FheadersImageCSV()

NameError: name 'validate_ticket' is not defined

In [ ]:
url = 'https://dilu.didi-food.com/gateway?wsisg='
global response
def stopEvents(eventIDList): #Batch stop promotions
	for country in eventIDList.keys():#,'CO','PE','CR']:
		#mids_c=mids[mids['country_code']==country]
		#eventsIDlist = list(pd.Series(mids_c['merchants_plan_id'].unique()).astype(str))
		print(f'{country} : {eventIDList[country]}')
		if len(eventIDList[country]) > 0:
			for i in range(1,int(len(eventIDList[country])/5) + 2) :
				datos={
					"api": "marketingIntl.merchantsPlan.stopShopAct",
					"country": country,
					"locale": "en-US",
					"merchantsPlanIDs": eventIDList[country][(i*5)-5 : i*5],
					"referer": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/list?actType=2",
					"ticket": ticket,
					"timezone": timeZone(country),
					"urlPath": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/list?actType=2"
					}
				response = requests.post(url, json=datos, headers=headersEvent)
				print(f'{((i*5)-5)+1} - {i*5} : {response.json()}')


In [ ]:
stopEvents({
    "PE": [
        "5764846706178852281",
        "5764846778937443773",
        "5764866219196744313",
        "5764866432523240036",
        "5764866265820627551",
        "5764866199521264229",
        "5764870731210886449",
        "5764890715777598189",
        "5764890807112762119",
        "5764890688418153219",
        "5764881159399867055",
        "5764881165548716719",
        "5764881180107145875",
        "5764881177674449583",
        "5764881035588207259",
        "5764881187849831080"
    ]
}
) #para usarse con la función stop batch promotions, usar el diccionario de notas

PE : ['5764846706178852281', '5764846778937443773', '5764866219196744313', '5764866432523240036', '5764866265820627551', '5764866199521264229', '5764870731210886449', '5764890715777598189', '5764890807112762119', '5764890688418153219', '5764881159399867055', '5764881165548716719', '5764881180107145875', '5764881177674449583', '5764881035588207259', '5764881187849831080']
1 - 5 : {'errno': 0, 'errmsg': 'ok', 'traceId': '0a9c305d675c8a1047b50aa209346302', 'time': 1734117904, 'data': {'successMerchantsPlanIDs': ['5764866265820627551'], 'failMerchantsPlanIDs': ['5764846706178852281', '5764846778937443773', '5764866219196744313', '5764866432523240036']}}
6 - 10 : {'errno': 0, 'errmsg': 'ok', 'traceId': '0a0f2a13675c8a1260d21bd9071d7002', 'time': 1734117906, 'data': {'successMerchantsPlanIDs': None, 'failMerchantsPlanIDs': ['5764866199521264229', '5764870731210886449', '5764890715777598189', '5764890807112762119', '5764890688418153219']}}
11 - 15 : {'errno': 0, 'errmsg': 'ok', 'traceId': '0a

In [ ]:
def stopEventsRegistration(eventIDList):
	for country in eventIDList.keys(): #batch stop registers
		#mids_c=mids[mids['country_code']==country]
		#eventsIDlist = list(pd.Series(mids_c['merchants_plan_id'].unique()).astype(str))
		print(f'{country} : {eventIDList[country]}')
		if len(eventIDList[country]) > 0:
			for i in range(0,len(eventIDList[country]))  :
				print([eventIDList[country][i]])
				datos={
					"api": "marketingIntl.merchantsPlan.stopShopJoin",
					"country": country,
					"locale": "en-US",
					"merchantsPlanIDs": [eventIDList[country][i]],
					"referer": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/list?actType=2",
					"ticket": ticket,
					"timezone": timeZone(country),
					"urlPath": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/list?actType=2"
					}
				response = requests.post(url, json=datos, headers=headersEvent)
				print(f'{i} : {response.json()}')
			print('\n')

In [ ]:
url = 'https://dilu.didi-food.com/gateway?wsisg='
def stopActivitiesOff(actshops): #batch to stop activities y necesita un data frame.
	global response
	global json_act_shops
	actshops['act_id']= actshops['act_id'].astype(str).str.strip()
	actshops['shop_id']= actshops['shop_id'].astype(str).str.strip()
	actshops['shop_id'] = actshops['shop_id'].str.lstrip('_')
	actshops['act_id'] = actshops['act_id'].str.lstrip('_')
	for country in actshops['锘緾ountry_code'].unique():
	#for country in countries:
		print(f'----------{country}----------')
		failNum = 0
		successNum = 0
		json_act_shops = actshops[actshops['锘緾ountry_code']== country][['shop_id','act_id']].drop_duplicates().rename(columns={"shop_id": "shopID", "act_id": "actID"}).to_dict(orient="records")
		num_promos_a_bajar = len(json_act_shops)
		print(num_promos_a_bajar )
		print(json_act_shops)
		if len(json_act_shops) > 0 :
			for i in range(1,int(len(json_act_shops)/10) + 2) :
				curr = json_act_shops[(i*10)-10 : i*10]
				#print(curr)
				if len(curr) > 0 :
					datos={
						"actShopList": curr,
						"actType": 2,
						"api": "marketingIntl.activity.stopShopAct",
						"country": country,
						"locale": "en-US",
						"referer": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/shopActivity/list?actType=2",
						"ticket": ticket,
						"timezone": timeZone(country),
						"urlPath": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/shopActivity/list?actType=2"
					}
					response = requests.post(url, json=datos, headers=headersEvent, timeout=120)
					data = response.json()
					failNum += data['data']['failNum']
					successNum += data['data']['successNum']

					print(f'Success Num: {successNum} , failNum: {failNum}, progress: {((successNum + failNum)*100) / num_promos_a_bajar} %  ')
		else:
				print('No hay restaurantes excedentes')

In [ ]:
ww10=pd.read_csv(r"C:\Users\DiDi\Documents\top_offenders.csv",encoding='GBK')
ww10


,锘緾ountry_code,shop_id,act_id
0,MX,5764609579770711294,5765406697969353237
1,MX,5764609852840870092,5765406700213305844
2,MX,5764609685416840332,5765406704759931380
3,MX,5764609639388548245,5765406779384988157
4,MX,5764609390032981453,5765406860456690199
5,MX,5764609542584011384,5765406738813485570
6,MX,5764609603992816046,5765406805045740013
7,MX,5764609676436834718,5765406608790061569
8,MX,5764609683101583770,5765406761110405638
9,MX,5764609493556791716,5765406621293281816


In [ ]:
stopActivitiesOff(ww10)
#stopActivitiesOff(df.iloc[::-1]) #sirve para que los detenga de abajo hacia arriba, en caso de que no se haya completado por error de conexión o algo así


----------MX----------
17
[{'shopID': '5764609579770711294', 'actID': '5765406697969353237'}, {'shopID': '5764609852840870092', 'actID': '5765406700213305844'}, {'shopID': '5764609685416840332', 'actID': '5765406704759931380'}, {'shopID': '5764609639388548245', 'actID': '5765406779384988157'}, {'shopID': '5764609390032981453', 'actID': '5765406860456690199'}, {'shopID': '5764609542584011384', 'actID': '5765406738813485570'}, {'shopID': '5764609603992816046', 'actID': '5765406805045740013'}, {'shopID': '5764609676436834718', 'actID': '5765406608790061569'}, {'shopID': '5764609683101583770', 'actID': '5765406761110405638'}, {'shopID': '5764609493556791716', 'actID': '5765406621293281816'}, {'shopID': '5764609459738118543', 'actID': '5765406768655958534'}, {'shopID': '5764609573114351281', 'actID': '5765406761139765775'}, {'shopID': '5764609603741157806', 'actID': '5765406768072950287'}, {'shopID': '5764609455535425954', 'actID': '5765406818811445757'}, {'shopID': '5764609682719903411', '

# Events Registration

In [46]:
import json
def getEventRegTemplate(act_name, actType, country, event_id, key, file_name, ticket):
    form = {
                    "actInfo": {
                        "actName": act_name,
                        "actType": actType
                    },
                    "api": "marketingIntl.merchantsPlan.createActByUploadFile",
                    "country": country,
                    "isBd": False,
                    "locale": "en-US",
                    "merchantsPlanInfo": {
                        "merchantsPlanID": event_id
                    },
                    "referer": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/list?actType=2",
                    "shopFilter": {
                        "gift": {
                            "giftDownloadUrl": f"https://img0.didiglobal.com/static/soda_public/{key}",
                            "giftKey": key,
                            "name": file_name
                        },
                        "shopFilterType": 0
                    },
                    "ticket": ticket,
                    "timezone": "America/Mexico_City",
                    "urlPath": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/list?actType=2"
                }
    return form


centralItemsSheet = 'Items Uploads'
ticket = validate_ticket()

url_top_upload_files_to_event = "https://dilu.didi-food.com/gateway?wsgsig=dd03-0Md9nWmm4JYQYC7S2N9iQsztIQ1SwXzS132VzcoUIQ1TZtKUJy//RsXl13YTZbCo7u6wOipS2vlop0Ni+yAhRifU18ryT03m24LPQbcSLuUSYg4uN4FVzCWW1JL"
headers_top_upload_files_to_event = {
    "Host": "dilu.didi-food.com",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:141.0) Gecko/20100101 Firefox/141.0",
    "Accept": "application/json",
    "Accept-Language": "es-MX,es;q=0.8,en-US;q=0.5,en;q=0.3",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Content-Type": "application/json",
    "Content-Length": "697",
    "Origin": "https://gattaran.didi-food.com",
    "Connection": "keep-alive",
    "Referer": "https://gattaran.didi-food.com/",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-site",
    "Pragma": "no-cache",
    "Cache-Control": "no-cache"
  }

# NUEVA FUNCIÓN: Para encontrar la fila de un evento por su ID
def findEventRowByID(event_id, sheet_name):
    sh = gc.open_by_url(urlGoogleSheet).worksheet(sheet_name)
    all_values = sh.get_all_values()
    df_temp = pd.DataFrame(all_values[1:], columns=all_values[0])

    # Buscar la fila donde el ID coincide
    matching_row = df_temp[df_temp['ID'] == event_id]

    if not matching_row.empty:
        # Devuelve la fila y el número de la fila original
        return matching_row.iloc[0], matching_row.index[0] + 2
    return None, None

# NUEVA FUNCIÓN: Para registrar el estado de la carga de ítems
def writeItemUploadStatus(row_number, upload_id, team):
    sh = gc.open_by_url(urlGoogleSheet).worksheet(centralItemsSheet)

    uploaded_column = 'M' if team=='Acquisition' else 'K'
    taskid_column = 'N' if team=='Acquisition' else 'L'
    # Actualiza la columna 'Uploaded' con el valor booleano True
    sh.update(f'{uploaded_column}{row_number}', [[True]])

    # Actualiza la columna 'Act IDs' con la lista de IDs de actividad
    sh.update(f'{taskid_column}{row_number}', [[upload_id]])

    #print(f"Cargados {num_items} ítems en {upload_id}")

# NUEVA FUNCIÓN: Para registrar el los resultados de la carga
def writeTaskResult(row_number, successShops, url, taskid, actType, successItems = 0, team = 'Engagement'):
    sh = gc.open_by_url(urlGoogleSheet).worksheet(centralItemsSheet)

    uploadedshops_column = 'K' if team=='Acquisition' else 'I'
    uploadeditems_column = 'L' if team=='Acquisition' else 'J'
    taskresult_column = 'O' if team=='Acquisition' else 'M'

    # Actualiza la columna 'Uploaded Shops' con el número de tiendas cargadas entre el numero de tiendas registradas
    sh.update(f'{uploadedshops_column}{row_number}', [[successShops]])

    if actType == 2 or actType == 4:
        # Actualiza la columna 'Uploaded Items' con el número de items cargados entre el numero de items registrados
        sh.update(f'{uploadeditems_column}{row_number}', [[successItems]])

    # Actualiza la columna 'Task Result' con el número de items cargados entre el numero de items registrados
    sh.update(f'{taskresult_column}{row_number}', [[url]])

    if actType == 2 or actType == 4:
        print(f"Cargados {successItems} ítems en {successShops} Rs en la Task {taskid}")
    else:
        print(f"Cargados {successShops} Rs en la Task {taskid}")

def uploadItems(team):
    global ticket
    global headersImageCSV
    global folderDriveShopLists
    global countries


    defineFoldersAndGSlink(team)
    urls_df=pd.DataFrame()
    
    items_events = SheetToDataframe(urlGoogleSheet, centralItemsSheet)

    for country in ['MX','CO','PE','CR']:
        urls_df=pd.concat([urls_df,pd.DataFrame.from_dict(requests.post(urlToUploadEvent, json=getFormTaskLists(country), headers=headersXLSX).json()['data']['taskInfos'])[['taskId','taskDownloadUrl']]])
       
    # Cargar solo la hoja de ítems centralizada
    for index, row in items_events.iterrows():
        rowNumber = index + 2
        readyToUpload = row.get('Ready to Upload', 'FALSE')
        act_name = row.get('Internal act name', '')
        country = row.get('Country', '')
        event_id = row.get('ID', '')
        activity_end_time = row.get('Event End Date', '')
        file_name = row.get('Templates', '')
        filename = f"{file_name}.xlsx" if not file_name.endswith('.xlsx') else file_name
        path = f"G:/Shared drives/Analytics_RCentral/Promos Automation/Engagement/Events/Templates/{filename}"
        # path = f"C:/Users/DiDi/Documents/Carga de combos/Combos Automation/{file_name}"  # ---- dry run
        items = row.get('Uploaded Items', '') # Ahora contendrá el número de items
        shops = row.get('Uploaded Shops', '') # Ahora contendrá el número de tiendas
        uploaded_flag = row.get('Uploaded', 'FALSE')
        taskid = row.get('Task ID','')
        result = row.get('Task Result','')
        eventType = row.get('Event Type','')
        actType = getActType(eventType)
        #print(eventType, actType)
        
        # CARGAR TEMPLATES
        if readyToUpload == 'TRUE' and event_id != '' and uploaded_flag == 'FALSE':
            print(f"Iniciando carga de ítems para el evento ID: {event_id}")
            global ticket
            ticket = validate_ticket()
            global headersImageCSV
            headersImageCSV = FheadersImageCSV()

            # Buscar datos del evento en las hojas originales
            event_data = None
            for event_sheet_name in sheets:
                event_data, _ = findEventRowByID(event_id, event_sheet_name)
                if event_data is not None:
                    break
            if event_data is None:
                print(f"Error: No se encontraron datos del evento {event_id} en las hojas originales. Omitiendo.")
                continue
            key = getKeyImageCSV(filename,items=True)
            if not key:
                print(f"Error al subir el template para el evento {event_id}. Omitiendo.")
                continue
            form_upload_items_to_event = getEventRegTemplate(act_name, actType, country, event_id, key, filename, ticket)
            response = requests.post(
                url_top_upload_files_to_event,
                json=form_upload_items_to_event,
                headers=headers_top_upload_files_to_event
            )
            upload_id=''
            response_json = response.json()
            print(response_json)
            if response_json['errmsg'] == 'ok' and 'data' in response_json:
                upload_id = response_json['data']
            if upload_id: # Si se generó upload id, entonces se actualiza la hoja
                writeItemUploadStatus(rowNumber, upload_id, team) #,num_items
                print(f"Todos los ítems para el evento {event_id} se cargaron con éxito.\n")

        # DESCARGAR TASK RESULT
        
        elif readyToUpload=='TRUE' and shops=='' and items in ('-','') and taskid!='' and uploaded_flag=='TRUE' and result =='' and not urls_df[urls_df['taskId']==taskid]['taskDownloadUrl'].reset_index(drop=True).empty:
            url=urls_df[urls_df['taskId']==taskid]['taskDownloadUrl'].reset_index(drop=True)[0]
            shopidcolumnName='Storeid'
            itemidcolumnName='Itemid'
            if '/img0.di' in url:
                print("El resultado para el evento  "+act_name+" es "+url)
                eventInfo=pd.read_csv(StringIO(str(requests.get(url).text).replace('\t','')))
                if actType == 2 or actType == 4:
                    writeTaskResult(rowNumber, str(eventInfo[eventInfo['Result']=='Yes'][shopidcolumnName].nunique()) + '/'+ str(eventInfo[shopidcolumnName].nunique()), url ,taskid, actType, str(eventInfo[eventInfo['Result']=='Yes'][itemidcolumnName].nunique())+ '/'+ str(eventInfo[itemidcolumnName].nunique()), team)
                else:
                    writeTaskResult(rowNumber, str(eventInfo[eventInfo['Result']=='Yes'][shopidcolumnName].nunique()) + '/'+ str(eventInfo[shopidcolumnName].nunique()), url ,taskid, actType, team=team)
                protectRow(rowNumber, centralItemsSheet)
                time.sleep(3)
    time.sleep(3)

In [47]:
urlToUploadEvent = 'https://dilu.didi-food.com/gateway?wsisg="dd03-1FjnbV9RWnOfHMZYEfpdLhhykgHa+O/Y0byC39HZkgH9JSE/emggNrBOVXO9J6MuBiu4MhF/VnYG3YYS0cXd3aAxV0rEK1EPFGjNLUEwV0kbHMesGDsD4r1yUnhB"'



def getFormTaskLists(country):
    formTaskLists={
        "api": "marketingIntl.task.getList",
        "country": country,
        "locale": "en-US",
        "page": 1,
        "pageSize": 200,
        "referer": "https://" + 'gattaran.didi-food.com'+"/v2/gtr_merchant-op/shopV2/toolsCenter/list",
        "status": -1,
        "ticket": ticket,
        "timezone": "America/Mexico_City",
        "urlPath": "https://"+ 'gattaran.didi-food.com'+ "/v2/gtr_merchant-op/shopV2/toolsCenter/list"
    }
    return formTaskLists

headersXLSX = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/113.0',
        'Accept': '*/*',
        'Accept-Language': 'es-MX,es;q=0.8,en-US;q=0.5,en;q=0.3',
        'Accept-Encoding': 'gzip, deflate, br',
        'ticket': ticket,
        'Connection': 'keep-alive',
    }


In [ ]:

while True:
    uploadItems(team)
    time.sleep(30) #Each minute upload the ready templates

-------------Engagement-------------
Shop Lists: G:\Shared drives\Analytics_RCentral\Promos Automation\Engagement\Events\XLSX
Images: G:\Shared drives\Analytics_RCentral\Promos Automation\Engagement\Events\Images
Templates: G:\Shared drives\Analytics_RCentral\Promos Automation\Engagement\Events\Templates
Images (My Drive): G:\My Drive\Images
Iniciando carga de ítems para el evento ID: 5765482361963611428
Error en shopids_toInt: 'utf-8' codec can't decode byte 0x95 in position 11: invalid start byte
do1_iMLuw6WnYGg1NGAjez95
{'errno': 0, 'errmsg': 'ok', 'traceId': '0a9e81d7695c09bf34d6e8a0079d1f02', 'time': 1767639488, 'data': '5765492164597321412'}


C:\Users\DiDi\AppData\Local\Temp\ipykernel_24224\4006151918.py:74: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sh.update(f'{uploaded_column}{row_number}', [[True]])
C:\Users\DiDi\AppData\Local\Temp\ipykernel_24224\4006151918.py:77: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sh.update(f'{taskid_column}{row_number}', [[upload_id]])


Todos los ítems para el evento 5765482361963611428 se cargaron con éxito.

Iniciando carga de ítems para el evento ID: 5765482222477837589
Error en shopids_toInt: 'utf-8' codec can't decode byte 0x95 in position 11: invalid start byte
do1_hPMzeMJayuld5SJHWWKG
{'errno': 0, 'errmsg': 'ok', 'traceId': '0a933b8b695c09cca7f7d8d207caad02', 'time': 1767639500, 'data': '5765492190169992935'}
Todos los ítems para el evento 5765482222477837589 se cargaron con éxito.

El resultado para el evento  BEng_CB_OFF_0126_off_100_CR es https://img0.didiglobal.com/static/soda_public/do1_zGIZXXiPj16h4JDqV2Ra


C:\Users\DiDi\AppData\Local\Temp\ipykernel_24224\4006151918.py:90: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sh.update(f'{uploadedshops_column}{row_number}', [[successShops]])
C:\Users\DiDi\AppData\Local\Temp\ipykernel_24224\4006151918.py:94: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sh.update(f'{uploadeditems_column}{row_number}', [[successItems]])
C:\Users\DiDi\AppData\Local\Temp\ipykernel_24224\4006151918.py:97: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sh.update(f'{taskresult_column}{row_number}', [[url]])


Cargados 0/49 ítems en 0/34 Rs en la Task 5765492332839243410
-------------Engagement-------------
Shop Lists: G:\Shared drives\Analytics_RCentral\Promos Automation\Engagement\Events\XLSX
Images: G:\Shared drives\Analytics_RCentral\Promos Automation\Engagement\Events\Images
Templates: G:\Shared drives\Analytics_RCentral\Promos Automation\Engagement\Events\Templates
Images (My Drive): G:\My Drive\Images
El resultado para el evento  BEng_CB_OFF_0126_off_100_MX es https://img0.didiglobal.com/static/soda_public/do1_dma57K6vBn5g91yzlyeP


...

Todos los ítems para el evento 5765482222477837589 se cargaron con éxito.

Iniciando carga de ítems para el evento ID: 5765492232410828330
Error en shopids_toInt: 'utf-8' codec can't decode byte 0x95 in position 11: invalid start byte
do1_6YWBiLr4WQDg46geelSp
{'errno': 0, 'errmsg': 'ok', 'traceId': '0a867063695c0ad93efceeff07de9402', 'time': 1767639769, 'data': '5765492303709802243'}
Todos los ítems para el evento 5765492232410828330 se cargaron con éxito.

-------------Engagement-------------
Shop Lists: G:\Shared drives\Analytics_RCentral\Promos Automation\Engagement\Events\XLSX
Images: G:\Shared drives\Analytics_RCentral\Promos Automation\Engagement\Events\Images
Templates: G:\Shared drives\Analytics_RCentral\Promos Automation\Engagement\Events\Templates
Images (My Drive): G:\My Drive\Images
El resultado para el evento  BEng_CB_OFF_0126_off_100_CO es https://img0.didiglobal.com/static/soda_public/do1_QjLt3L5l7MYIILHh6JUv


C:\Users\DiDi\AppData\Local\Temp\ipykernel_24224\4006151918.py:90: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sh.update(f'{uploadedshops_column}{row_number}', [[successShops]])
C:\Users\DiDi\AppData\Local\Temp\ipykernel_24224\4006151918.py:94: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sh.update(f'{uploadeditems_column}{row_number}', [[successItems]])
C:\Users\DiDi\AppData\Local\Temp\ipykernel_24224\4006151918.py:97: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sh.update(f'{taskresult_column}{row_number}', [[url]])


Cargados 4/6 ítems en 4/6 Rs en la Task 5765492303781105388
El resultado para el evento  BEng_CB_OFF_0126_off_100_CR es https://img0.didiglobal.com/static/soda_public/do1_hAPzQ3Uy5ughBlZbKQl8


C:\Users\DiDi\AppData\Local\Temp\ipykernel_24224\4006151918.py:90: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sh.update(f'{uploadedshops_column}{row_number}', [[successShops]])
C:\Users\DiDi\AppData\Local\Temp\ipykernel_24224\4006151918.py:94: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sh.update(f'{uploadeditems_column}{row_number}', [[successItems]])
C:\Users\DiDi\AppData\Local\Temp\ipykernel_24224\4006151918.py:97: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sh.update(f'{taskresult_column}{row_number}', [[url]])


Cargados 1/1 ítems en 1/1 Rs en la Task 5765492303709802243


# Drafts

In [ ]:
{
	"activityMode": 1,
	"actSubsidyTargetType": 0,
	"actType": 2,
	"api": "marketingIntl.common.checkFileFormat",
	"country": "MX",
	"giftUrl": "https://img0.didiglobal.com/static/soda_public/do1_iAGvycgE5JRPe4I58NjR",
	"gtrNewLocale": "en_MX",
	"locale": "en-US",
	"merchantsPlanID": "5765302362245629600",
	"referer": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/list?actType=2",
	"specialItemType": 1,
	"templateType": 5,
	"ticket": "5de42c6542883447980ff42524e482980002200166000",
	"timezone": "America/Mexico_City",
	"urlPath": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/list?actType=2"
}

{
	"actInfo": {
		"actName": "¡Premio Combos Nuevos!",
		"actType": 2
	},
	"api": "marketingIntl.merchantsPlan.createActByUploadFile",
	"country": "MX",
	"gtrNewLocale": "en_MX",
	"isBd": false,
	"locale": "en-US",
	"merchantsPlanInfo": {
		"merchantsPlanID": "5765302362245629600"
	},
	"referer": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/list?actType=2",
	"shopFilter": {
		"gift": {
			"giftDownloadUrl": "https://img0.didiglobal.com/static/soda_public/do1_iAGvycgE5JRPe4I58NjR",
			"giftKey": "do1_iAGvycgE5JRPe4I58NjR",
			"name": "combotest4.xlsx"
		},
		"shopFilterType": 0
	},
	"ticket": "5de42c6542883447980ff42524e482980002200166000",
	"timezone": "America/Mexico_City",
	"urlPath": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/list?actType=2"
}

In [ ]:
# SxSy
{
	"actInfo": {
		"actName": "Test_AutomatizaciónRegistros",
		"actType": 1
	},
	"api": "marketingIntl.merchantsPlan.createActByUploadFile",
	"country": "MX",
	"gtrNewLocale": "en_MX",
	"isBd": false,
	"locale": "en-US",
	"merchantsPlanInfo": {
		"merchantsPlanID": "5765332197680546698"
	},
	"referer": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/list?actType=1",
	"shopFilter": {
		"gift": {
			"giftDownloadUrl": "https://img0.didiglobal.com/static/soda_public/do1_C5l4BiRMUZQv4e425UsQ",
			"giftKey": "do1_C5l4BiRMUZQv4e425UsQ",
			"name": "test2 (1).xlsx"
		},
		"shopFilterType": 0
	},
	"ticket": "99a98509c212949ad6c7764f4ef36d220002200166000",
	"timezone": "America/Mexico_City",
	"urlPath": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/list?actType=1"
}

NameError: name 'false' is not defined

In [ ]:
# FD

{
	"actInfo": {
		"actName": "Test_AtuomatizaciónRegistros",
		"actType": 5
	},
	"api": "marketingIntl.merchantsPlan.createActByUploadFile",
	"country": "MX",
	"gtrNewLocale": "en_MX",
	"isBd": false,
	"locale": "en-US",
	"merchantsPlanInfo": {
		"merchantsPlanID": "5765332135000867880"
	},
	"referer": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/list?actType=5",
	"shopFilter": {
		"gift": {
			"giftDownloadUrl": "https://img0.didiglobal.com/static/soda_public/do1_32ZiK5dBpnGhVArPCPdu",
			"giftKey": "do1_32ZiK5dBpnGhVArPCPdu",
			"name": "testfd (3).xlsx"
		},
		"shopFilterType": 0
	},
	"ticket": "99a98509c212949ad6c7764f4ef36d220002200166000",
	"timezone": "America/Mexico_City",
	"urlPath": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/list?actType=5"
}

{
	"actInfo": {
		"actDays": 0,
		"actEndTime": "2025-10-17 23:59:59",
		"actName": "prueba offline gattaran",
		"actStartTime": "2025-10-16 00:00:00",
		"actTimeType": 2,
		"actType": 2,
		"internalActName": "prueba off",
		"periodSoldInfo": {
			"dayType": [
				{
					"duration": {
						"begin": "00:00",
						"end": "24:00"
					}
				}
			],
			"soldInfoType": 1,
			"weekType": null
		},
		"specialItemRule": [
			{
				"checkItemPriceDay": null,
				"itemCrowdType": 0,
				"itemPromoRangeValue": null,
				"itemType": null,
				"picLimit": 1,
				"priceLimit": {
					"max": 250,
					"min": 15
				},
				"rulePurposeType": 0,
				"rules": [
					{
						"content": {
							"discountType": 0,
							"discountValueList": [],
							"discountValueRange": {
								"max": 50,
								"min": 25
							}
						},
						"type": 1
					}
				],
				"selectItemNumRange": {
					"max": 5,
					"min": 3
				}
			}
		],
		"userType": 1
	},
	"api": "marketing-mis.marketing.merchantsPlan.create",
	"country": "MX",
	"gtrNewLocale": "en_MX",
	"locale": "en-US",
	"merchantsPlanInfo": {
		"actFlowType": 0,
		"activityMode": 0,
		"actSceneType": 7,
		"actType": 2,
		"auditUserList": [
			"oscarregueiralopez"
		],
		"brandId": "",
		"brandName": "",
		"countyGroupIDs": [
			"520901000001",
			"521516000001"
		],
		"joinLimitCityData": {
			"52010100": {
				"cityName": "Aguascalientes",
				"joinNumber": 75,
				"limitNumber": 0
			},
			"52020100": {
				"cityName": "Mexicali",
				"joinNumber": 99,
				"limitNumber": 0
			},
			"52020200": {
				"cityName": "Tijuana",
				"joinNumber": 73,
				"limitNumber": 0
			},
			"52020300": {
				"cityName": "Ensenada",
				"joinNumber": 52,
				"limitNumber": 0
			},
			"52030200": {
				"cityName": "La Paz",
				"joinNumber": 41,
				"limitNumber": 0
			},
			"52030300": {
				"cityName": "Los Cabos",
				"joinNumber": 35,
				"limitNumber": 0
			},
			"52040100": {
				"cityName": "Campeche",


			},
			"52070900": {
",
				"joinNumber": 87,
				"limitNumber": 0
			},
			"52090100": {
				"cityName": "Mexico City",
				"joinNumber": 458,
				"limitNumber": 0
			},
			"52100200": {
				"cityName": "Durango",
				"joinNumber": 23,
				"limitNumber": 0
			},
			"52110200": {
				"cityName": "Celaya",
				"joinNumber": 3,
				"limitNumber": 0
			},
			"52110300": {
				"cityName": "Manuel Doblado",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52110400": {
				"cityName": "Irapuato",
				"joinNumber": 29,
				"limitNumber": 0
			},
			"52110500": {
				"cityName": "Leon",
				"joinNumber": 40,
				"limitNumber": 0
			},
			"52110600": {
				"cityName": "Salamanca",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52110800": {
				"cityName": "San Miguel de Allende",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52120200": {
				"cityName": "Acapulco",
				"joinNumber": 5,
				"limitNumber": 0
			},
			"52120500": {
				"cityName": "Chilpancingo de los Bravo",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52131100": {
				"cityName": "Pachuca",
				"joinNumber": 2,
				"limitNumber": 0
			},
			"52131500": {
				"cityName": "Tulancingo",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52140500": {
				"cityName": "Guadalajara",
				"joinNumber": 299,
				"limitNumber": 0
			},
			"52140900": {
				"cityName": "Vallarta",
				"joinNumber": 29,
				"limitNumber": 0
			},
			"52151600": {
				"cityName": "Toluca",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52160200": {
				"cityName": "La Piedad",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52160400": {
				"cityName": "Morelia",
				"joinNumber": 80,
				"limitNumber": 0
			},
			"52160800": {
				"cityName": "Uruapan",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52160900": {
				"cityName": "Zamora",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52170200": {
				"cityName": "Cuernavaca(Cuautla)",
				"joinNumber": 2,
				"limitNumber": 0
			},
			"52180200": {
				"cityName": "Tepic",
				"joinNumber": 47,
				"limitNumber": 0
			},
			"52190500": {
				"cityName": "Monterrey",
				"joinNumber": 8,
				"limitNumber": 0
			},
			"52200100": {
				"cityName": "Oaxaca",
				"joinNumber": 33,
				"limitNumber": 0
			},
			"52210400": {
				"cityName": "Puebla",
				"joinNumber": 68,
				"limitNumber": 0
			},
			"52210600": {
				"cityName": "Tehuacan",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52220200": {
				"cityName": "San Juan del Río",
				"joinNumber": 1,
				"limitNumber": 0
			},
			"52220400": {
				"cityName": "Queretaro",
				"joinNumber": 86,
				"limitNumber": 0
			},
			"52230100": {
				"cityName": "Cancun",
				"joinNumber": 65,
				"limitNumber": 0
			},
			"52230200": {
				"cityName": "Quintana Roo",
				"joinNumber": 1,
				"limitNumber": 0
			},
			"52230300": {
				"cityName": "Chetumal",
				"joinNumber": 7,
				"limitNumber": 0
			},
			"52240100": {
				"cityName": "Valles City",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52240400": {
				"cityName": "San Luis Potosi",
				"joinNumber": 62,
				"limitNumber": 0
			},
			"52250100": {
				"cityName": "Culiacán",
				"joinNumber": 77,
				"limitNumber": 0
			},
			"52250300": {
				"cityName": "Mazatlan",
				"joinNumber": 49,
				"limitNumber": 0
			},
			"52250400": {
				"cityName": "Los Mochis-Guasave",
				"joinNumber": 33,
				"limitNumber": 0
			},
			"52260100": {
				"cityName": "Ciudad Obregón",
				"joinNumber": 32,
				"limitNumber": 0
			},
			"52260200": {
				"cityName": "Hermosillo",
				"joinNumber": 41,
				"limitNumber": 0
			},
			"52260300": {
				"cityName": "Navojoa",
				"joinNumber": 3,
				"limitNumber": 0
			},
			"52260400": {
				"cityName": "Nogales(SON)",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52260900": {
				"cityName": "Guaymas",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52270100": {
				"cityName": "Villahermosa",
				"joinNumber": 3,
				"limitNumber": 0
			},
			"52280100": {
				"cityName": "Nuevo Laredo",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52280200": {
				"cityName": "Reynosa",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52280300": {
				"cityName": "Tampico",
				"joinNumber": 1,
				"limitNumber": 0
			},
			"52280400": {
				"cityName": "Victoria",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52280500": {
				"cityName": "Matamoros",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52290400": {
				"cityName": "Tlaxcala",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52300200": {
				"cityName": "Coatzacoalcos",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52300600": {
				"cityName": "Poza Rica de Hidalgo",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52300900": {
				"cityName": "Veracruz",
				"joinNumber": 1,
				"limitNumber": 0
			},
			"52301000": {
				"cityName": "Xalapa",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52301200": {
				"cityName": "Cordoba",
				"joinNumber": 30,
				"limitNumber": 0
			},
			"52301300": {
				"cityName": "Orizaba",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52302300": {
				"cityName": "Minatitlán",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52310300": {
				"cityName": "Merida",
				"joinNumber": 35,
				"limitNumber": 0
			},
			"52320100": {
				"cityName": "Zacatecas",
				"joinNumber": 16,
				"limitNumber": 0
			}
		},
		"joinLimitNumber": 0,
		"joinLimitType": 0,
		"merchantsPlanEndTime": "2025-10-10 23:59:59",
		"MerchantsPlanEndTimeOffsetInfo": {
			"offset": -21600,
			"timestamp": 1759125539
		},
		"merchantsPlanID": "5765315465788198452",
		"merchantsPlanMode": 1,
		"merchantsPlanStartTime": "2025-10-09 00:00:00",
		"merchantsPlanStartTimeOffsetInfo": {
			"offset": -21600,
			"timestamp": 1757656800
		},
		"merchantsPlanType": 1,
		"presentPlatformList": [
			4
		],
		"radioFlowType": 0,
		"recommend": 0,
		"status": 7
	},
	"player": "nevermore",
	"push": {
		"introText": "-",
		"popImgs": null,
		"pushChannel": [],
		"pushText": "-"
	},
	"referer": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/add?signUpId=5765315465788198452&actType=2&isFromCopy=1",
	"shopFilter": {
		"gift": {},
		"shopFilterType": 3,
		"shopIds": [],
		"tag": {
			"groupName": "invitados_dummy_aPA",
			"tagFileUrl": "https://img0.didiglobal.com/static/soda_public/do1_xb4jLJgSJVT7Bd98NKsM",
			"tagId": "658429",
			"updateType": 1,
			"userTypeName": "shop"
		}
	},
	"subsidyBudget": {
		"actPlatformBudgetLimit": 9999999999,
		"actSharePlatformBudgetPeriodDayLimit": 0,
		"actShopBudgetLimit": 0,
		"actShopBudgetThreshold": {
			"enable": true,
			"threshold": 9879870
		},
		"actSubsidyLevelRule": {},
		"actSubsidyLimit": 0,
		"actSubsidyList": [],
		"actSubsidyRange": {
			"max": 25,
			"min": 5
		},
		"actSubsidyTargetType": 5,
		"isCopy": true,
		"subsidyAmount": 0
	},
	"ticket": "e4fb1d64749ee68e8f1d2e39067366e00002200166000",
	"timezone": "America/Mexico_City",
	"urlPath": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/add?signUpId=5765315465788198452&actType=2&isFromCopy=1"
}

In [ ]:
{
	"actInfo": {
		"actDays": 0,
		"actEndTime": "2025-10-17 23:59:59",
		"actName": "prueba offline gattaran",
		"actStartTime": "2025-10-16 00:00:00",
		"actTimeType": 2,
		"actType": 2,
		"internalActName": "prueba off",
		"periodSoldInfo": {
			"dayType": [
				{
					"duration": {
						"begin": "00:00",
						"end": "24:00"
					}
				}
			],
			"soldInfoType": 1,
			"weekType": null
		},
		"specialItemRule": [
			{
				"checkItemPriceDay": null,
				"itemCrowdType": 0,
				"itemPromoRangeValue": null,
				"itemType": null,
				"picLimit": 1,
				"priceLimit": {
					"max": 250,
					"min": 15
				},
				"rulePurposeType": 0,
				"rules": [
					{
						"content": {
							"discountType": 0,
							"discountValueList": [],
							"discountValueRange": {
								"max": 50,
								"min": 25
							}
						},
						"type": 1
					}
				],
				"selectItemNumRange": {
					"max": 5,
					"min": 3
				}
			}
		],
		"userType": 1
	},
	"api": "marketing-mis.marketing.merchantsPlan.create",
	"country": "MX",
	"gtrNewLocale": "en_MX",
	"locale": "en-US",
	"merchantsPlanInfo": {
		"actFlowType": 0,
		"activityMode": 0,
		"actSceneType": 7,
		"actType": 2,
		"auditUserList": [
			"oscarregueiralopez"
		],
		"brandId": "",
		"brandName": "",
		"countyGroupIDs": [
			"520101000001",
			"521102000001",
			"522601000001",
			"520601000001",
			"521702000001",
			"522501000001",
			"521002000001",
			"520203000001",
			"521405000001",
			"522609000001",
			"522602000001",
			"521104000001",
			"520302000001",
			"521602000001",
			"521105000001",
			"520303000001",
			"522504000001",
			"521103000001",
			"520602000001",
			"522503000001",
			"520201000001",
			"521604000001",
			"522603000001",
			"522604000001",
			"521311000001",
			"522104000001",
			"522204000001",
			"521106000001",
			"522202000001",
			"521108000001",
			"521802000001",
			"520202000001",
			"522904000001",
			"521315000001",
			"521608000001",
			"521409000001",
			"523201000001",
			"521609000001"
		],
		"joinLimitCityData": {

		},
		"joinLimitNumber": 0,
		"joinLimitType": 0,
		"merchantsPlanEndTime": "2025-10-10 23:59:59",
		"MerchantsPlanEndTimeOffsetInfo": {
			"offset": -21600,
			"timestamp": 1760162399
		},
		"merchantsPlanID": "5765351491852242770",
		"merchantsPlanMode": 1,
		"merchantsPlanStartTime": "2025-10-09 00:00:00",
		"merchantsPlanStartTimeOffsetInfo": {
			"offset": -21600,
			"timestamp": 1759989600
		},
		"merchantsPlanType": 1,
		"presentPlatformList": [
			2
		],
		"radioFlowType": 0,
		"recommend": 0,
		"status": 1
	},
	"player": "nevermore",
	"push": {
		"introText": "-",
		"popImgs": null,
		"pushChannel": [],
		"pushText": "-"
	},
	"referer": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/add?signUpId=5765351491852242770&actType=2&isFromCopy=1",
	"shopFilter": {
		"gift": {},
		"shopFilterType": 3,
		"shopIds": [],
		"tag": {
			"groupName": "invitados_dummy_aPA",
			"tagFileUrl": "https://img0.didiglobal.com/static/soda_public/do1_KbicfsAjwSunTSGYZX2u",
			"tagId": "658429",
			"updateType": 1,
			"userTypeName": "shop"
		}
	},
	"subsidyBudget": {
		"actPlatformBudgetLimit": 9999999999,
		"actSharePlatformBudgetPeriodDayLimit": 0,
		"actShopBudgetLimit": 0,
		"actShopBudgetThreshold": {
			"enable": false,
			"threshold": 0
		},
		"actSubsidyLevelRule": {},
		"actSubsidyLimit": 0,
		"actSubsidyList": [],
		"actSubsidyRange": {
			"max": 25,
			"min": 5
		},
		"actSubsidyTargetType": 5,
		"isCopy": true,
		"subsidyAmount": 0
	},
	"ticket": "e4fb1d64749ee68e8f1d2e39067366e00002200166000",
	"timezone": "America/Mexico_City",
	"urlPath": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/add?signUpId=5765351491852242770&actType=2&isFromCopy=1"
}

In [ ]:
{
	"actInfo": {
		"actDays": 0,
		"actEndTime": "2025-10-17 23:59:59",
		"actName": "prueba offline gattaran",
		"actStartTime": "2025-10-16 00:00:00",
		"actTimeType": 2,
		"actType": 2,
		"internalActName": "prueba off",
		"periodSoldInfo": {
			"dayType": [
				{
					"duration": {
						"begin": "00:00",
						"end": "24:00"
					}
				}
			],
			"soldInfoType": 1,
			"weekType": null
		},
		"specialItemRule": [
			{
				"checkItemPriceDay": null,
				"itemCrowdType": 0,
				"itemPromoRangeValue": null,
				"itemType": null,
				"picLimit": 1,
				"priceLimit": {
					"max": 250,
					"min": 15
				},
				"rulePurposeType": 0,
				"rules": [
					{
						"content": {
							"discountType": 0,
							"discountValueList": [],
							"discountValueRange": {
								"max": 50,
								"min": 25
							}
						},
						"type": 1
					}
				],
				"selectItemNumRange": {
					"max": 5,
					"min": 3
				}
			}
		],
		"userType": 1
	},
	"api": "marketing-mis.marketing.merchantsPlan.create",
	"country": "MX",
	"gtrNewLocale": "en_MX",
	"locale": "en-US",
	"merchantsPlanInfo": {
		"actFlowType": 0,
		"activityMode": 0,
		"actSceneType": 7,
		"actType": 2,
		"auditUserList": [
			"oscarregueiralopez"
		],
		"brandId": "",
		"brandName": "",
		"countyGroupIDs": [
			"520101000001",
			"521102000001",
			"522601000001",
			"520601000001",
			"521702000001",
			"522501000001",
			"521002000001",
			"520203000001",
			"521405000001",
			"522609000001",
			"522602000001",
			"521104000001",
			"520302000001",
			"521602000001",
			"521105000001",
			"520303000001",
			"522504000001",
			"521103000001",
			"520602000001",
			"522503000001",
			"520201000001",
			"521604000001",
			"522603000001",
			"522604000001",
			"521311000001",
			"522104000001",
			"522204000001",
			"521106000001",
			"522202000001",
			"521108000001",
			"521802000001",
			"520202000001",
			"522904000001",
			"521315000001",
			"521608000001",
			"521409000001",
			"523201000001",
			"521609000001"
		],
		"joinLimitCityData": {
			"52010100": {
				"cityName": "Aguascalientes",
				"joinNumber": 0,
				"limitNumber": 0
		},
		"joinLimitNumber": 0,
		"joinLimitType": 0,
		"merchantsPlanEndTime": "2025-10-10 23:59:59",
		"MerchantsPlanEndTimeOffsetInfo": {
			"offset": -21600,
			"timestamp": 1760162399
		},
		"merchantsPlanID": "5765351624748761178",
		"merchantsPlanMode": 1,
		"merchantsPlanStartTime": "2025-10-09 00:00:00",
		"merchantsPlanStartTimeOffsetInfo": {
			"offset": -21600,
			"timestamp": 1759989600
		},
		"merchantsPlanType": 1,
		"presentPlatformList": [
			2,
			4
		],
		"radioFlowType": 0,
		"recommend": 1,
		"status": 9
	},
	"player": "nevermore",
	"push": {
		"introText": "-",
		"popImgs": null,
		"pushChannel": [],
		"pushText": "-"
	},
	"referer": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/add?signUpId=5765351624748761178&actType=2&isFromCopy=1",
	"shopFilter": {
		"gift": {},
		"shopFilterType": 3,
		"shopIds": [],
		"tag": {
			"groupName": "invitados_dummy_aPA",
			"tagFileUrl": "https://img0.didiglobal.com/static/soda_public/do1_Zr9jTxu8e8j7Uyfqz53R",
			"tagId": "658429",
			"updateType": 1,
			"userTypeName": "shop"
		}
	},
	"subsidyBudget": {
		"actPlatformBudgetLimit": 9999999999,
		"actSharePlatformBudgetPeriodDayLimit": 0,
		"actShopBudgetLimit": 0,
		"actShopBudgetThreshold": {
			"enable": false,
			"threshold": 0
		},
		"actSubsidyLevelRule": {},
		"actSubsidyLimit": 0,
		"actSubsidyList": [],
		"actSubsidyRange": {
			"max": 25,
			"min": 5
		},
		"actSubsidyTargetType": 5,
		"isCopy": true,
		"subsidyAmount": 0
	},
	"ticket": "e4fb1d64749ee68e8f1d2e39067366e00002200166000",
	"timezone": "America/Mexico_City",
	"urlPath": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/add?signUpId=5765351624748761178&actType=2&isFromCopy=1"
}

In [ ]:
{
	"actInfo": {
		"actDays": 0,
		"actEndTime": "2025-11-17 23:59:59",
		"actName": "ST Buen Fin ",
		"actStartTime": "2025-11-14 00:00:00",
		"actTimeType": 1,
		"actType": 2,
		"internalActName": "BEng_C_OFF_1125_off_10-50_MX_BuenFin",
		"periodSoldInfo": {
			"dayType": [],
			"soldInfoType": 1,
			"weekType": null
		},
		"specialItemRule": [
			{
				"checkItemPriceDay": 7,
				"itemCrowdType": 0,
				"itemPromoRangeValue": null,
				"itemType": 1,
				"picLimit": 1,
				"priceLimit": {
					"max": 250,
					"min": 99
				},
				"rulePurposeType": 0,
				"rules": [
					{
						"content": {
							"discountType": 1,
							"discountValueList": [
								99
							],
							"discountValueRange": null
						},
						"type": 3
					}
				],
				"selectItemNumRange": {
					"max": 2,
					"min": 1
				}
			}
		],
		"userType": 0
	},
	"api": "marketing-mis.marketing.merchantsPlan.create",
	"country": "MX",
	"gtrNewLocale": "en_MX",
	"locale": "en-US",
	"merchantsPlanInfo": {
		"actFlowType": 0,
		"activityMode": 0,
		"actSceneType": 9,
		"auditUserList": [
			"mirenguagnelligarcia"
		],
		"countyGroupIDs": [
			"520901000001",
			"521516000001",
			"521101000001",
			"521202000001",
			"520401000001",
			"522301000001",
			"520402000001",
			"522303000001",
			"520802000001",
			"521205000001",
			"523002000001",
			"520702000001",
			"523012000001",
			"520804000001",
			"520808000001",
			"522805000001",
			"523103000001",
			"523023000001",
			"520502000001",
			"521905000001",
			"522801000001",
			"522001000001",
			"523013000001",
			"520603000001",
			"520503000001",
			"522302000001",
			"523006000001",
			"522802000001",
			"520505000001",
			"520708000001",
			"522404000001",
			"522803000001",
			"520709000001",
			"522106000001",
			"520504000001",
			"520711000001",
			"522401000001",
			"523009000001",
			"522804000001",
			"522701000001",
			"523010000001",
			"520101000001",
			"521102000001",
			"522601000001",
			"520601000001",
			"521702000001",
			"522501000001",
			"521002000001",
			"520203000001",
			"521405000001",
			"522609000001",
			"522602000001",
			"521104000001",
			"520302000001",
			"521602000001",
			"521105000001",
			"520303000001",
			"522504000001",
			"521103000001",
			"520602000001",
			"522503000001",
			"520201000001",
			"521604000001",
			"522603000001",
			"522604000001",
			"521311000001",
			"522104000001",
			"522204000001",
			"521106000001",
			"522202000001",
			"521108000001",
			"521802000001",
			"520202000001",
			"522904000001",
			"521315000001",
			"521608000001",
			"521409000001",
			"523201000001",
			"521609000001"
		],
		"joinLimitType": 0,
		"merchantsPlanEndTime": "2025-11-01 23:59:59",
		"merchantsPlanMode": null,
		"merchantsPlanStartTime": "2025-10-10 00:00:00",
		"merchantsPlanType": 1,
		"presentPlatformList": [
			4,
			2
		],
		"radioFlowType": 0,
		"recommend": 0
	},
	"player": "nevermore",
	"push": {
		"introText": "-",
		"popImgs": null,
		"pushChannel": [],
		"pushText": "-"
	},
	"referer": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/add?actType=2",
	"shopFilter": {
		"shopFilterType": 3,
		"tag": {
			"groupName": "MX_Buenfin_1025",
			"tagFileUrl": "https://img0.didiglobal.com/static/soda_public/do1_c71vqmqCmwxc5YRq9Nir",
			"tagId": "658649",
			"updateType": 1,
			"userTypeName": "shop"
		}
	},
	"subsidyBudget": {
		"actPlatformBudgetLimit": 2,
		"actSharePlatformBudgetPeriodDayLimit": 1,
		"actShopBudgetLimit": 0,
		"actShopBudgetThreshold": {
			"enable": false,
			"threshold": 0
		},
		"actSubsidyLevelRule": {},
		"actSubsidyLimit": 0,
		"actSubsidyList": [
			{
				"subsidyAmount": 0
			}
		],
		"actSubsidyRange": {
			"max": 50,
			"min": 10
		},
		"actSubsidyTargetType": 5,
		"factorType": 0,
		"isCopy": false,
		"subsidyAmount": 0
	},
	"ticket": "d2d93c1adb0190c2e08f46ef924543140002200166000",
	"timezone": "America/Mexico_City",
	"urlPath": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/add?actType=2"
}

In [ ]:
# dcoupon

{
	"actInfo": {
		"actDays": 0,
		"actEndTime": "2026-02-08 23:59:59",
		"actName": "Test D-Coupon",
		"actStartTime": "2026-01-12 00:00:00",
		"actTimeType": 1,
		"actType": 13,
		"internalActName": "test_dcoupon",
		"periodSoldInfo": {
			"dayType": [],
			"soldInfoType": 1,
			"weekType": null
		},
		"unionCouponRule": {
			"content": {
				"minStockPerDay": 1,
				"platSubsidyText": "este es un texto :)))",
				"rules": [
					{
						"discount": 20,
						"threshold": 0
					}
				],
				"suggestStockPerDay": 1
			},
			"type": 1
		},
		"userType": 0
	},
	"api": "marketing-mis.marketing.merchantsPlan.create",
	"country": "MX",
	"gtrNewLocale": "en_MX",
	"locale": "en-US",
	"merchantsPlanInfo": {
		"actFlowType": 0,
		"activityMode": 0,
		"actSceneType": 16,
		"auditUserList": [
			""
		],
		"countyGroupIDs": [
			"520901000001",
			"521516000001",
			"521101000001",
			"521202000001",
			"520401000001",
			"522301000001",
			"520402000001",
			"522303000001",
			"520802000001",
			"521205000001",
			"523002000001",
			"520702000001",
			"523012000001",
			"520804000001",
			"520808000001",
			"522805000001",
			"523103000001",
			"523023000001",
			"520502000001",
			"521905000001",
			"522801000001",
			"522001000001",
			"523013000001",
			"520603000001",
			"520503000001",
			"523006000001",
			"522302000001",
			"522802000001",
			"520505000001",
			"520708000001",
			"522404000001",
			"522803000001",
			"520709000001",
			"522106000001",
			"520504000001",
			"520711000001",
			"522401000001",
			"523009000001",
			"522804000001",
			"522701000001",
			"523010000001",
			"520101000001",
			"521102000001",
			"522601000001",
			"520601000001",
			"521702000001",
			"522501000001",
			"521002000001",
			"520203000001",
			"521405000001",
			"522609000001",
			"522602000001",
			"521104000001",
			"520302000001",
			"521602000001",
			"521105000001",
			"520303000001",
			"522504000001",
			"521103000001",
			"520602000001",
			"522503000001",
			"520201000001",
			"521604000001",
			"522603000001",
			"522604000001",
			"521311000001",
			"522104000001",
			"522204000001",
			"521106000001",
			"522202000001",
			"521108000001",
			"521802000001",
			"520202000001",
			"522904000001",
			"521315000001",
			"521608000001",
			"521409000001",
			"523201000001",
			"521609000001",
			"521599000002",
			"521599000001"
		],
		"joinLimitType": 0,
		"merchantsPlanEndTime": "2026-01-11 23:59:59",
		"merchantsPlanMode": null,
		"merchantsPlanStartTime": "2025-12-18 00:00:00",
		"merchantsPlanType": 1,
		"presentPlatformList": [
			2,
			4,
			1
		],
		"radioFlowType": 0,
		"recommend": 0
	},
	"player": "nevermore",
	"push": {
		"introText": "descripcion del evento :)))",
		"popImgs": null,
		"pushChannel": [
			"home"
		],
		"pushText": "textooooooo :))"
	},
	"referer": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/add?actType=13",
	"shopFilter": {
		"shopFilterType": 3,
		"tag": {
			"groupName": "test_dummy",
			"tagFileUrl": "https://img0.didiglobal.com/static/soda_public/do1_rZERzefDHhzSX57Yah1Q",
			"tagId": "663061",
			"updateType": 1,
			"userTypeName": "shop"
		}
	},
	"subsidyBudget": {
		"actPlatformBudgetLimit": 0,
		"actSharePlatformBudgetPeriodDayLimit": 0,
		"actShopBudgetLimit": 0,
		"actSubsidyLevelRule": {},
		"actSubsidyLimit": 0,
		"actSubsidyList": [
			{
				"subsidyAmount": 0
			}
		],
		"actSubsidyRange": {
			"max": 0,
			"min": 0
		},
		"actSubsidyTargetType": 0,
		"factorType": 0,
		"subsidyAmount": 0
	},
	"ticket": "3c0bba4eb5f27d309dc7d83099c2735c0002200166000",
	"timezone": "America/Mexico_City",
	"urlPath": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/add?actType=13"
}

In [ ]:
# system transfer lista de invitados

{
	"actInfo": {
		"actDays": 2,
		"actEndTime": "",
		"actName": "SUP 2 Hunting SBT",
		"actStartTime": "",
		"actTimeType": 1,
		"actType": 2,
		"internalActName": "SUP2_Hunting_SBT_Test",
		"periodSoldInfo": {
			"dayType": [],
			"soldInfoType": 1,
			"weekType": null
		},
		"specialItemRule": [
			{
				"checkItemPriceDay": 7,
				"itemCrowdType": 0,
				"itemPromoRangeValue": null,
				"itemType": null,
				"picLimit": 1,
				"priceLimit": {
					"max": 270,
					"min": 20
				},
				"rulePurposeType": 0,
				"rules": [
					{
						"content": {
							"discountType": 1,
							"discountValueList": [
								25
							],
							"discountValueRange": null
						},
						"type": 1
					}
				],
				"selectItemNumRange": {
					"max": 5,
					"min": 1
				}
			}
		],
		"userType": 0
	},
	"api": "marketing-mis.marketing.merchantsPlan.create",
	"country": "MX",
	"gtrNewLocale": "en_MX",
	"locale": "en-US",
	"merchantsPlanInfo": {
		"actFlowType": 0,
		"activityMode": 0,
		"actSceneType": 6,
		"actType": 2,
		"auditUserList": [
			"mirenguagnelligarcia"
		],
		"brandId": "",
		"brandName": "",
		"countyGroupIDs": [
			"520901000001",
			"521516000001"
		],
		"joinLimitCityData": {
			"52090100": {
				"cityName": "Mexico City",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52151600": {
				"cityName": "Toluca",
				"joinNumber": 0,
				"limitNumber": 0
			}
		},
		"joinLimitNumber": 0,
		"joinLimitType": 0,
		"merchantsPlanEndTime": "2026-01-11 23:59:59",
		"MerchantsPlanEndTimeOffsetInfo": {
			"offset": -21600,
			"timestamp": 1768024799
		},
		"merchantsPlanID": "5765494717548528867",
		"merchantsPlanMode": 1,
		"merchantsPlanStartTime": "2026-01-08 00:00:00",
		"merchantsPlanStartTimeOffsetInfo": {
			"offset": -21600,
			"timestamp": 1767765600
		},
		"merchantsPlanType": 2,
		"operatorName": "svelascogarcia_i",
		"presentPlatformList": [
			4
		],
		"radioFlowType": 0,
		"recommend": 0,
		"status": 1,
		"statusDesc": "Awaiting review"
	},
	"player": "nevermore",
	"push": {
		"introText": "Sigamos creciendo juntos SUP 2",
		"popImgs": null,
		"pushChannel": [],
		"pushText": "Querido socio restaurantero, gracias a tu buen desempeño en la plataforma tu restaurante ha sido seleccionado para ser parte de una campaña exclusiva de promoción con subsidiada por DiDi Food, para ofrecer un 30% de descuento en 5 de tus platillos más vendidos y aumentar tu visibilidad en la app, que se convertirá en más órdenes para tu restaurante. SUP 2"
	},
	"referer": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/add?signUpId=5765494717548528867&actType=2&isFromCopy=1",
	"shopFilter": {
		"gift": {},
		"shopFilterType": 4,
		"shopIds": [],
		"tag": {}
	},
	"subsidyBudget": {
		"actPlatformBudgetLimit": 10000000,
		"actSharePlatformBudgetPeriodDayLimit": 0,
		"actShopBudgetLimit": 6500,
		"actSubsidyLevelRule": {},
		"actSubsidyLimit": 0,
		"actSubsidyList": [
			{
				"subsidyAmount": 85
			}
		],
		"actSubsidyRange": {
			"max": 0,
			"min": 0
		},
		"actSubsidyTargetType": 0,
		"isCopy": true,
		"subsidyAmount": 85
	},
	"ticket": "a5d0db8496e988c3535da89c356f14590002200166000",
	"timezone": "America/Mexico_City",
	"urlPath": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/add?signUpId=5765494717548528867&actType=2&isFromCopy=1"
}

In [ ]:
{
	"actInfo": {
		"actDays": 0,
		"actEndTime": "2026-01-23 23:59:59",
		"actName": "¡Ofrece descuentos! F1TNB",
		"actStartTime": "2026-01-08 00:00:00",
		"actTimeType": 2,
		"actType": 2,
		"internalActName": "MX_Test_Eng_Core_2602_AP_OFF_NewU_260107_Test_men",
		"periodSoldInfo": {
			"dayType": [
				{
					"duration": {
						"begin": "00:00",
						"end": "24:00"
					}
				}
			],
			"soldInfoType": 1,
			"weekType": null
		},
		"specialItemRule": [
			{
				"checkItemPriceDay": null,
				"itemCrowdType": 0,
				"itemPromoRangeValue": null,
				"itemType": null,
				"picLimit": 1,
				"priceLimit": {
					"max": 250,
					"min": 10
				},
				"rulePurposeType": 0,
				"rules": [
					{
						"content": {
							"discountType": 0,
							"discountValueList": [],
							"discountValueRange": {
								"max": 50,
								"min": 25
							}
						},
						"type": 1
					}
				],
				"selectItemNumRange": {
					"max": 4,
					"min": 3
				}
			}
		],
		"userType": 1
	},
	"api": "marketing-mis.marketing.merchantsPlan.create",
	"country": "MX",
	"gtrNewLocale": "en_MX",
	"locale": "en-US",
	"merchantsPlanInfo": {
		"actFlowType": 0,
		"activityMode": 0,
		"actSceneType": 3,
		"actType": 2,
		"auditUserList": [
			"oscarregueiralopez"
		],
		"brandId": "",
		"brandName": "",
		"countyGroupIDs": [
			"520901000001",
			"521516000001"
		],
		"joinLimitCityData": {
			"52010100": {
				"cityName": "Aguascalientes",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52020100": {
				"cityName": "Mexicali",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52020200": {
				"cityName": "Tijuana",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52020300": {
				"cityName": "Ensenada",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52030200": {
				"cityName": "La Paz",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52030300": {
				"cityName": "Los Cabos",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52040100": {
				"cityName": "Campeche",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52040200": {
				"cityName": "Carmen",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52050200": {
				"cityName": "Monclova",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52050300": {
				"cityName": "Piedras Negras",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52050400": {
				"cityName": "Torreon",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52050500": {
				"cityName": "Saltillo",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52060100": {
				"cityName": "Colima",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52060200": {
				"cityName": "Manzanillo",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52060300": {
				"cityName": "Peña Colorada",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52070200": {
				"cityName": "Comitan de Dominguez",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52070800": {
				"cityName": "San Cristobal de las Casas",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52070900": {
				"cityName": "Tapachula",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52071100": {
				"cityName": "Tuxtla Gutierrez",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52080200": {
				"cityName": "Chihuahua",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52080400": {
				"cityName": "Delicias",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52080800": {
				"cityName": "Juarez(CHIH)",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52090100": {
				"cityName": "Mexico City",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52100200": {
				"cityName": "Durango",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52110200": {
				"cityName": "Celaya",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52110300": {
				"cityName": "Manuel Doblado",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52110400": {
				"cityName": "Irapuato",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52110500": {
				"cityName": "Leon",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52110600": {
				"cityName": "Salamanca",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52110800": {
				"cityName": "San Miguel de Allende",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52120200": {
				"cityName": "Acapulco",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52120500": {
				"cityName": "Chilpancingo de los Bravo",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52131100": {
				"cityName": "Pachuca",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52131500": {
				"cityName": "Tulancingo",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52140500": {
				"cityName": "Guadalajara",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52140900": {
				"cityName": "Vallarta",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52151600": {
				"cityName": "Toluca",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52160200": {
				"cityName": "La Piedad",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52160400": {
				"cityName": "Morelia",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52160800": {
				"cityName": "Uruapan",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52160900": {
				"cityName": "Zamora",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52170200": {
				"cityName": "Cuernavaca(Cuautla)",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52180200": {
				"cityName": "Tepic",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52190500": {
				"cityName": "Monterrey",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52200100": {
				"cityName": "Oaxaca",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52210400": {
				"cityName": "Puebla",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52210600": {
				"cityName": "Tehuacan",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52220200": {
				"cityName": "San Juan del Río",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52220400": {
				"cityName": "Queretaro",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52230100": {
				"cityName": "Cancun",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52230200": {
				"cityName": "Quintana Roo",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52230300": {
				"cityName": "Chetumal",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52240100": {
				"cityName": "Valles City",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52240400": {
				"cityName": "San Luis Potosi",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52250100": {
				"cityName": "Culiacán",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52250300": {
				"cityName": "Mazatlan",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52250400": {
				"cityName": "Los Mochis-Guasave",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52260100": {
				"cityName": "Ciudad Obregón",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52260200": {
				"cityName": "Hermosillo",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52260300": {
				"cityName": "Navojoa",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52260400": {
				"cityName": "Nogales(SON)",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52260900": {
				"cityName": "Guaymas",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52270100": {
				"cityName": "Villahermosa",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52280100": {
				"cityName": "Nuevo Laredo",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52280200": {
				"cityName": "Reynosa",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52280300": {
				"cityName": "Tampico",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52280400": {
				"cityName": "Victoria",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52280500": {
				"cityName": "Matamoros",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52290400": {
				"cityName": "Tlaxcala",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52300200": {
				"cityName": "Coatzacoalcos",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52300600": {
				"cityName": "Poza Rica de Hidalgo",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52300900": {
				"cityName": "Veracruz",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52301000": {
				"cityName": "Xalapa",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52301200": {
				"cityName": "Cordoba",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52301300": {
				"cityName": "Orizaba",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52302300": {
				"cityName": "Minatitlán",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52310300": {
				"cityName": "Merida",
				"joinNumber": 0,
				"limitNumber": 0
			},
			"52320100": {
				"cityName": "Zacatecas",
				"joinNumber": 0,
				"limitNumber": 0
			}
		},
		"joinLimitNumber": 0,
		"joinLimitType": 0,
		"merchantsPlanEndTime": "2026-01-13 23:59:59",
		"MerchantsPlanEndTimeOffsetInfo": {
			"offset": -21600,
			"timestamp": 1770011939
		},
		"merchantsPlanID": "5765495873100251290",
		"merchantsPlanMode": 1,
		"merchantsPlanStartTime": "2026-01-08 00:00:00",
		"merchantsPlanStartTimeOffsetInfo": {
			"offset": -21600,
			"timestamp": 1767765600
		},
		"merchantsPlanType": 1,
		"operatorName": "oscarregueiralopez",
		"presentPlatformList": [
			2,
			4
		],
		"radioFlowType": 0,
		"recommend": 0,
		"status": 6,
		"statusDesc": "Active"
	},
	"player": "nevermore",
	"push": {
		"introText": "eeeeeeee",
		"popImgs": null,
		"pushChannel": [],
		"pushText": "eeeeeeeeeee"
	},
	"referer": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/add?signUpId=5765495873100251290&actType=2&isFromCopy=1",
	"shopFilter": {
		"gift": {},
		"shopFilterType": 3,
		"shopIds": [],
		"tag": {
			"groupName": "invitados_dumm_zfr",
			"tagFileUrl": "https://img0.didiglobal.com/static/soda_public/do1_uvt5IJLsdfY2WyQs1GVc",
			"tagId": "664099",
			"updateType": 1,
			"userTypeName": "shop"
		}
	},
	"subsidyBudget": {
		"actPlatformBudgetLimit": 0,
		"actSharePlatformBudgetPeriodDayLimit": 0,
		"actShopBudgetLimit": 0,
		"actShopBudgetThreshold": {
			"enable": false,
			"threshold": 0
		},
		"actSubsidyLevelRule": {},
		"actSubsidyLimit": 0,
		"actSubsidyList": [
			{
				"subsidyAmount": 0
			}
		],
		"actSubsidyRange": {
			"max": 0,
			"min": 0
		},
		"actSubsidyTargetType": 0,
		"isCopy": true,
		"subsidyAmount": 0
	},
	"ticket": "bf65658f41b1f880aa2833cee7417e650002200166000",
	"timezone": "America/Mexico_City",
	"urlPath": "https://gattaran.didi-food.com/v2/gtr_merchant-op/shopV2/signUp/add?signUpId=5765495873100251290&actType=2&isFromCopy=1"
}